Pour ce projet, nous avons suivi le schéma présenté dans cet article:
https://towardsdatascience.com/a-complete-machine-learning-walk-through-in-python-part-one-c62152f39420

# Introduction: Home Credit Default Risk Competition
-----------------

## Data

Les données sont fournies par [Home Credit] (http://www.homecredit.net/about-us.aspx), un service dédié à la fourniture de lignes de crédit (prêts) à la population non bancarisée. Prédire si un client remboursera ou non un prêt ou s'il rencontrera des difficultés est un besoin commercial essentiel, et Home Credit organise ce concours sur Kaggle pour voir quel type de modèles la communauté de l'apprentissage automatique peut développer pour l'aider dans cette tâche. 

Il existe 7 sources de données différentes :


* **application_train/application_test**: the main training and testing data avec des informations sur chaque demande de prêt chez Home Credit. Chaque prêt a sa propre ligne et est identifié par la caractéristique `SK_ID_CURR`. Les données du train sont accompagnées de la  `TARGET` qui indique 0 : le prêt a été remboursé ou 1 : le prêt n'a pas été remboursé. 
* **bureau**: les données concernant les crédits antérieurs du client auprès d'autres institutions financières. Chaque crédit antérieur a sa propre ligne dans le bureau, mais un prêt dans les données de la demande peut avoir plusieurs crédits antérieurs.
* **bureau_balance**: données mensuelles sur les crédits antérieurs dans le bureau. Chaque ligne correspond à un mois d'un crédit antérieur, et un seul crédit antérieur peut avoir plusieurs lignes, une pour chaque mois de la durée du crédit. 
* **previous_application**: les demandes de crédit antérieures auprès de Home Credit. Chaque prêt en cours dans les données de la demande peut avoir plusieurs prêts antérieurs. Chaque demande antérieure a une ligne et est identifiée par la caractéristique `SK_ID_PREV`. 
* **POS_CASH_BALANCE**: des données mensuelles sur les prêts au point de vente ou au comptant que les clients ont contractés auprès de Home Credit. Chaque ligne correspond à un mois d'un prêt au point de vente ou d'un prêt au comptant antérieur, et un seul prêt antérieur peut avoir plusieurs lignes.
* **credit_card_balance**: des données mensuelles sur les cartes de crédit que les clients ont eues précédemment avec Home Credit. Chaque ligne correspond à un mois de solde de carte de crédit, et une seule carte de crédit peut avoir plusieurs lignes.
* **installments_payment** l'historique des paiements pour les prêts précédents chez Home Credit. Il y a une ligne pour chaque paiement effectué et une ligne pour chaque paiement manqué. 



Voir schéma tables pour assemblage : "C:\Users\belou\Projet 7\input\schéma_tables.png"
Voir dico variables : "C:\Users\belou\Projet 7\input\dico_variables.xlsx"

## Metric: ROC AUC

La mesure de classification courante utilisée c'est la COURBE ROC :

https://stats.stackexchange.com/questions/132777/what-does-auc-stand-for-and-what-is-it
https://en.wikipedia.org/wiki/Receiver_operating_characteristic
https://darwin.unmc.edu/dxtests/roc3.htm


Lorsqu'on mesure une classification en fonction de l'AUC, on ne génère pas des prédictions 0 ou 1, mais plutôt une probabilité entre 0 et 1. Cela peut être déroutant car nous aimons généralement penser en termes de précision, mais lorsque nous rencontrons des problèmes avec des classes déséquilibrées (nous verrons que c'est le cas), la précision n'est pas la meilleure métrique. Par exemple, si je voulais construire un modèle capable de détecter les terroristes avec une précision de 99,9999 %, je me contenterais de créer un modèle prédisant que chaque personne n'est pas un terroriste. Il est clair que cela ne serait pas efficace (le rappel serait nul) et nous utilisons des mesures plus avancées telles que la SSC ROC ou le score F1 pour refléter plus précisément les performances d'un classificateur. 

https://en.wikipedia.org/wiki/F-score

Un modèle ayant un ROC AUC élevé aura également une grande précision, mais l'AUC est une meilleure représentation de la performance du modèle.

https://datascience.stackexchange.com/questions/806/advantages-of-auc-vs-standard-accuracy


# 1 - Exploration des datasets Train et Test
--------------

Dans ce NB, nous nous en tiendrons à utiliser uniquement les données d'entraînement et de test de l'application principale. Bien que nous devions utiliser toutes les données si nous voulions avoir un espoir de rivaliser sérieusement, nous nous en tiendrons pour l'instant à un seul fichier, ce qui devrait être plus facile à gérer. Cela nous permettra d'établir une base de référence que nous pourrons ensuite améliorer. Dans ce genre de projet, il est préférable de comprendre le problème petit à petit plutôt que de s'y plonger complètement et de s'y perdre ! 

## 1.1 - Imports


In [1]:
import mlflow
from PIL import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer, roc_auc_score, f1_score, fbeta_score, recall_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

import lightgbm as lgb

from mlflow.models import infer_signature

import shap
# Memory management
import gc 

# featuretools for automated feature engineering
import featuretools as ft


## 1.2 - Read in Dataset

In [2]:
# Read in the datasets and limit to the first 1000 rows (sorted by SK_ID_CURR) 
# This allows us to actually see the results in a reasonable amount of time! 
app_train = pd.read_csv('application_train.csv').sort_values('SK_ID_CURR').reset_index(drop = True).loc[:1000, :]
bureau = pd.read_csv('bureaux.csv').sort_values(['SK_ID_CURR', 'SK_ID_BUREAU']).reset_index(drop = True).loc[:1000, :]
bureau_balance = pd.read_csv('bureau_balances.csv').sort_values('SK_ID_BUREAU').reset_index(drop = True).loc[:1000, :]
cash = pd.read_csv('POS_CASH_balances.csv').sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index(drop = True).loc[:1000, :]
credit = pd.read_csv('credit_card_balances.csv').sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index(drop = True).loc[:1000, :]
previous = pd.read_csv('previous_applications.csv').sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index(drop = True).loc[:1000, :]
installments = pd.read_csv('installments_payment.csv').sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index(drop = True).loc[:1000, :]

**Principes de base de Featuretools**
Featuretools est une bibliothèque Python open-source qui permet de créer automatiquement des caractéristiques à partir d'un ensemble de tables connexes en utilisant une technique appelée « deep feature synthesis » (synthèse profonde de caractéristiques). L'ingénierie automatisée des caractéristiques, comme beaucoup de sujets dans l'apprentissage automatique, est un sujet complexe construit sur une base d'idées plus simples. En examinant ces idées une par une, nous pouvons développer notre compréhension de featuretools, ce qui nous permettra plus tard d'en tirer le meilleur parti.

**Entités et ensembles d'entités**
Une entité est simplement une table ou, dans Pandas, un dataframe. Les observations se trouvent dans les lignes et les caractéristiques dans les colonnes. Une entité dans featuretools doit avoir un index unique où aucun des éléments n'est dupliqué. Actuellement, seuls app, bureau et previous ont des index uniques (SK_ID_CURR, SK_ID_BUREAU et SK_ID_PREV respectivement). Pour les autres dataframe, nous devons passer make_index = True, puis spécifier le nom de l'index. Les entités peuvent également avoir des indices de temps où chaque entrée est identifiée par un temps unique. (Il n'y a pas de date dans les données, mais il y a des temps relatifs, donnés en mois ou en jours, que nous pourrions envisager de traiter comme des variables temporelles).

Un EntitySet est un ensemble de tables et de relations entre elles. Il peut être considéré comme une structure de données avec ses propres méthodes et attributs. L'utilisation d'un EntitySet nous permet de regrouper plusieurs tables et de les manipuler beaucoup plus rapidement que des tables individuelles.

Tout d'abord, nous allons créer un entityset vide nommé clients pour garder une trace de toutes les données.

In [34]:
# Initialisation d'EntitySet
es = ft.EntitySet(id='clients')

In [35]:
es

Entityset: clients
  DataFrames:
  Relationships:
    No relationships

Nous définissons maintenant chaque entité, ou table de données. Nous devons passer un index si les données en ont un ou make_index = True si ce n'est pas le cas. Featuretools déduit automatiquement les types de variables, mais nous pouvons également les modifier si nécessaire. Par exemple, si nous avons une variable catégorielle qui est représentée par un entier, nous pourrions souhaiter que Featuretools connaisse le bon type.

In [36]:
# Adding dataframes
es.add_dataframe(dataframe_name='app_train', dataframe=app_train, index='SK_ID_CURR')
es.add_dataframe(dataframe_name='bureau', dataframe=bureau, index='SK_ID_BUREAU')
es.add_dataframe(dataframe_name='previous', dataframe=previous, index='SK_ID_PREV')
es.add_dataframe(dataframe_name='bureau_balance', dataframe=bureau_balance, make_index=True, index='bureaubalance_index')
es.add_dataframe(dataframe_name='cash', dataframe=cash, make_index=True, index='cash_index')
es.add_dataframe(dataframe_name='credit', dataframe=credit, make_index=True, index='credit_index')
es.add_dataframe(dataframe_name='installments', dataframe=installments, make_index=True, index='installments_index')



Entityset: clients
  DataFrames:
    app_train [Rows: 1001, Columns: 122]
    bureau [Rows: 1001, Columns: 17]
    previous [Rows: 1001, Columns: 37]
    bureau_balance [Rows: 1001, Columns: 4]
    cash [Rows: 1001, Columns: 9]
    credit [Rows: 1001, Columns: 24]
    installments [Rows: 1001, Columns: 9]
  Relationships:
    No relationships

In [37]:
es

Entityset: clients
  DataFrames:
    app_train [Rows: 1001, Columns: 122]
    bureau [Rows: 1001, Columns: 17]
    previous [Rows: 1001, Columns: 37]
    bureau_balance [Rows: 1001, Columns: 4]
    cash [Rows: 1001, Columns: 9]
    credit [Rows: 1001, Columns: 24]
    installments [Rows: 1001, Columns: 9]
  Relationships:
    No relationships

**Relations**
Les relations sont un concept fondamental non seulement dans les featuretools, mais aussi dans toute base de données relationnelle. La meilleure façon d'appréhender une relation de type « un à plusieurs » est de faire l'analogie avec la relation « parent à enfant ». Un parent est un individu unique, mais il peut avoir plusieurs enfants. Les enfants peuvent ensuite avoir plusieurs enfants à leur tour. Dans une table parent, chaque individu a une seule ligne. Chaque individu de la table parent peut avoir plusieurs lignes dans la table enfant.

Par exemple, le daatframe de l'application comporte une ligne pour chaque client (SK_ID_CURR), tandis que le dataframe du bureau comporte plusieurs prêts antérieurs (SK_ID_PREV) pour chaque parent (SK_ID_CURR). Par conséquent, le dataframe bureau est l'enfant du dataframe app. Le dataframe bureau est à son tour le parent de bureau_balance car chaque prêt a une ligne dans bureau mais plusieurs enregistrements mensuels dans bureau_balance.

In [38]:
print('Parent: app_train, Parent Variable: SK_ID_CURR\n\n', app_train.iloc[:, 111:115].head())
print('\nChild: bureau, Child Variable: SK_ID_CURR\n\n', bureau.iloc[10:30, :4].head())

Parent: app_train, Parent Variable: SK_ID_CURR

         FLAG_DOCUMENT_17  FLAG_DOCUMENT_18  FLAG_DOCUMENT_19  FLAG_DOCUMENT_20
100002                 0                 0                 0                 0
100003                 0                 0                 0                 0
100004                 0                 0                 0                 0
100006                 0                 0                 0                 0
100007                 0                 0                 0                 0

Child: bureau, Child Variable: SK_ID_CURR

          SK_ID_CURR  SK_ID_BUREAU CREDIT_ACTIVE CREDIT_CURRENCY
6158905      100002       6158905        Closed      currency 1
6158906      100002       6158906        Closed      currency 1
6158907      100002       6158907        Closed      currency 1
6158908      100002       6158908        Closed      currency 1
6158909      100002       6158909        Active      currency 1


Le SK_ID_CURR « 100002 » a une ligne dans la table parent et plusieurs lignes dans la table enfant.

Deux tables sont liées par une variable partagée. Les dataframe app et bureau sont liés par la variable SK_ID_CURR, tandis que les dataframe bureau et bureau_balance sont liés par la variable SK_ID_BUREAU. La définition des relations est relativement simple et le diagramme fourni par le concours est utile pour visualiser les relations. Pour chaque relation, nous devons spécifier la variable parent et la variable enfant. Au total, il existe six relations entre les tables. Ci-dessous, nous spécifions les six relations et les ajoutons à l'EntitySet.

In [39]:
# Define relationships using the correct parameters for the Relationship constructor
r_app_bureau = ft.Relationship(
    entityset=es,
    parent_dataframe_name='app_train', 
    parent_column_name='SK_ID_CURR', 
    child_dataframe_name='bureau', 
    child_column_name='SK_ID_CURR'
)
r_bureau_balance = ft.Relationship(
    entityset=es,
    parent_dataframe_name='bureau', 
    parent_column_name='SK_ID_BUREAU', 
    child_dataframe_name='bureau_balance', 
    child_column_name='SK_ID_BUREAU'
)
r_app_previous = ft.Relationship(
    entityset=es,
    parent_dataframe_name='app_train', 
    parent_column_name='SK_ID_CURR', 
    child_dataframe_name='previous', 
    child_column_name='SK_ID_CURR'
)
r_previous_cash = ft.Relationship(
    entityset=es,
    parent_dataframe_name='previous', 
    parent_column_name='SK_ID_PREV', 
    child_dataframe_name='cash', 
    child_column_name='SK_ID_PREV'
)
r_previous_installments = ft.Relationship(
    entityset=es,
    parent_dataframe_name='previous', 
    parent_column_name='SK_ID_PREV', 
    child_dataframe_name='installments', 
    child_column_name='SK_ID_PREV'
)
r_previous_credit = ft.Relationship(
    entityset=es,
    parent_dataframe_name='previous', 
    parent_column_name='SK_ID_PREV', 
    child_dataframe_name='credit', 
    child_column_name='SK_ID_PREV'
)


In [40]:
# Add relationships to the EntitySet
es.add_relationships([r_app_bureau, r_bureau_balance, r_app_previous, r_previous_cash, r_previous_installments, r_previous_credit])

Entityset: clients
  DataFrames:
    app_train [Rows: 1001, Columns: 122]
    bureau [Rows: 1001, Columns: 17]
    previous [Rows: 1001, Columns: 37]
    bureau_balance [Rows: 1001, Columns: 4]
    cash [Rows: 1001, Columns: 9]
    credit [Rows: 1001, Columns: 24]
    installments [Rows: 1001, Columns: 9]
  Relationships:
    bureau.SK_ID_CURR -> app_train.SK_ID_CURR
    bureau_balance.SK_ID_BUREAU -> bureau.SK_ID_BUREAU
    previous.SK_ID_CURR -> app_train.SK_ID_CURR
    cash.SK_ID_PREV -> previous.SK_ID_PREV
    installments.SK_ID_PREV -> previous.SK_ID_PREV
    credit.SK_ID_PREV -> previous.SK_ID_PREV

In [41]:
es

Entityset: clients
  DataFrames:
    app_train [Rows: 1001, Columns: 122]
    bureau [Rows: 1001, Columns: 17]
    previous [Rows: 1001, Columns: 37]
    bureau_balance [Rows: 1001, Columns: 4]
    cash [Rows: 1001, Columns: 9]
    credit [Rows: 1001, Columns: 24]
    installments [Rows: 1001, Columns: 9]
  Relationships:
    bureau.SK_ID_CURR -> app_train.SK_ID_CURR
    bureau_balance.SK_ID_BUREAU -> bureau.SK_ID_BUREAU
    previous.SK_ID_CURR -> app_train.SK_ID_CURR
    cash.SK_ID_PREV -> previous.SK_ID_PREV
    installments.SK_ID_PREV -> previous.SK_ID_PREV
    credit.SK_ID_PREV -> previous.SK_ID_PREV

Remarque un peu plus poussée : nous devons veiller à ne pas créer un graphique en diamant où il y a plusieurs chemins d'un parent à un enfant. Si nous relions directement l'application et l'argent via SK_ID_CURR, le précédent et l'argent via SK_ID_PREV, et l'application et le précédent via SK_ID_CURR, nous avons créé deux chemins de l'application à l'argent. Il en résulte une ambiguïté, de sorte que l'approche que nous devons adopter consiste à relier app à cash par l'intermédiaire de previous. Nous établissons une relation entre previous (le parent) et cash (l'enfant) en utilisant SK_ID_PREV. Nous établissons ensuite une relation entre app (le parent) et previous (maintenant l'enfant) à l'aide de SK_ID_CURR. Ensuite, featuretools sera en mesure de créer des fonctionnalités sur app dérivées de previous et cash en empilant plusieurs primitives.

Toutes les entités de l'entité peuvent être liées les unes aux autres. En théorie, cela nous permet de calculer des caractéristiques pour n'importe laquelle des entités, mais en pratique, nous ne calculerons des caractéristiques que pour le dataframe app puisqu'il sera utilisé pour la formation/le test.

**Feature Primitives**
Une primitive de caractéristique est une opération appliquée à une table ou à un ensemble de tables pour créer une caractéristique. Il s'agit de calculs simples, dont beaucoup sont déjà utilisés dans l'ingénierie manuelle des caractéristiques, qui peuvent être empilés les uns sur les autres pour créer des caractéristiques complexes. Les primitives d'entités se répartissent en deux catégories :

- **Agrégation** : fonction qui regroupe les points de données enfants pour chaque parent et calcule ensuite une statistique telle que la moyenne, le minimum, le maximum ou l'écart type. Un exemple est le calcul du montant maximum du prêt précédent pour chaque client. Une agrégation fonctionne sur plusieurs tables en utilisant les relations entre les tables.
- **Transformation** : opération appliquée à une ou plusieurs colonnes d'une même table. Un exemple serait de prendre la valeur absolue d'une colonne, ou de trouver la différence entre deux colonnes dans un tableau.
Une liste des primitives disponibles dans featuretools peut être consultée ci-dessous.

In [42]:
# List the primitives in a dataframe
primitives = ft.list_primitives()
pd.options.display.max_colwidth = 100
primitives[primitives['type'] == 'aggregation'].head(10)

,name,type,description,valid_inputs,return_type
0,count_below_mean,aggregation,Determines the number of values that are below the mean.,<ColumnSchema (Semantic Tags = ['numeric'])>,<ColumnSchema (Logical Type = IntegerNullable) (Semantic Tags = ['numeric'])>
1,has_no_duplicates,aggregation,Determines if there are duplicates in the input.,"<ColumnSchema (Semantic Tags = ['category'])>, <ColumnSchema (Semantic Tags = ['numeric'])>",<ColumnSchema (Logical Type = BooleanNullable)>
2,num_false_since_last_true,aggregation,Calculates the number of 'False' values since the last `True` value. Description: From a series ...,<ColumnSchema (Logical Type = Boolean)>,<ColumnSchema (Logical Type = IntegerNullable) (Semantic Tags = ['numeric'])>
3,any,aggregation,Determines if any value is 'True' in a list.,"<ColumnSchema (Logical Type = Boolean)>, <ColumnSchema (Logical Type = BooleanNullable)>",<ColumnSchema (Logical Type = Boolean)>
4,max_consecutive_positives,aggregation,Determines the maximum number of consecutive positive values in the input,"<ColumnSchema (Logical Type = Integer)>, <ColumnSchema (Logical Type = Double)>",<ColumnSchema (Logical Type = Integer) (Semantic Tags = ['numeric'])>
5,percent_true,aggregation,Determines the percent of `True` values.,"<ColumnSchema (Logical Type = Boolean)>, <ColumnSchema (Logical Type = BooleanNullable)>",<ColumnSchema (Logical Type = Double) (Semantic Tags = ['numeric'])>
6,all,aggregation,Calculates if all values are 'True' in a list.,"<ColumnSchema (Logical Type = Boolean)>, <ColumnSchema (Logical Type = BooleanNullable)>",<ColumnSchema (Logical Type = Boolean)>
7,count_greater_than,aggregation,Determines the number of values greater than a controllable threshold.,<ColumnSchema (Semantic Tags = ['numeric'])>,<ColumnSchema (Logical Type = Integer) (Semantic Tags = ['numeric'])>
8,n_unique_days,aggregation,Determines the number of unique days.,<ColumnSchema (Logical Type = Datetime)>,<ColumnSchema (Logical Type = Integer) (Semantic Tags = ['numeric'])>
9,is_monotonically_increasing,aggregation,Determines if a series is monotonically increasing.,<ColumnSchema (Semantic Tags = ['numeric'])>,<ColumnSchema (Logical Type = BooleanNullable)>


In [43]:
primitives[primitives['type'] == 'transform'].head(10)

,name,type,description,valid_inputs,return_type
65,cum_sum,transform,Calculates the cumulative sum.,<ColumnSchema (Semantic Tags = ['numeric'])>,<ColumnSchema (Semantic Tags = ['numeric'])>
66,exponential_weighted_std,transform,Computes the exponentially weighted moving standard deviation for a series of numbers,<ColumnSchema (Semantic Tags = ['numeric'])>,<ColumnSchema (Logical Type = Double) (Semantic Tags = ['numeric'])>
67,percent_change,transform,Determines the percent difference between values in a list.,<ColumnSchema (Semantic Tags = ['numeric'])>,<ColumnSchema (Logical Type = Double) (Semantic Tags = ['numeric'])>
68,week,transform,Determines the week of the year from a datetime.,<ColumnSchema (Logical Type = Datetime)>,"<ColumnSchema (Logical Type = Ordinal: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 1..."
69,age,transform,Calculates the age in years as a floating point number given a date of birth.,<ColumnSchema (Logical Type = Datetime) (Semantic Tags = ['date_of_birth'])>,<ColumnSchema (Logical Type = AgeFractional) (Semantic Tags = ['numeric'])>
70,number_of_words_in_quotes,transform,Determines the number of words in quotes in a string.,<ColumnSchema (Logical Type = NaturalLanguage)>,<ColumnSchema (Logical Type = IntegerNullable) (Semantic Tags = ['numeric'])>
71,not,transform,Negates a boolean value.,"<ColumnSchema (Logical Type = Boolean)>, <ColumnSchema (Logical Type = BooleanNullable)>",<ColumnSchema (Logical Type = BooleanNullable)>
72,modulo_numeric_scalar,transform,Computes the modulo of each element in the list by a given scalar.,<ColumnSchema (Semantic Tags = ['numeric'])>,<ColumnSchema (Semantic Tags = ['numeric'])>
73,diff_datetime,transform,Computes the timedelta between a datetime in a list and the previous datetime in that list.,<ColumnSchema (Logical Type = Datetime)>,<ColumnSchema (Logical Type = Timedelta)>
74,rolling_trend,transform,Calculates the trend of a given window of entries of a column over time.,"<ColumnSchema (Logical Type = Datetime) (Semantic Tags = ['time_index'])>, <ColumnSchema (Semant...",<ColumnSchema (Logical Type = Double) (Semantic Tags = ['numeric'])>


Synthèse approfondie des fonctionnalités
La synthèse profonde des fonctionnalités (DFS) est le processus utilisé par featuretools pour créer de nouvelles fonctionnalités. La DFS empile les primitives des caractéristiques pour former des caractéristiques dont la « profondeur » est égale au nombre de primitives. Par exemple, si nous prenons la valeur maximale des prêts précédents d'un client (disons MAX(previous.loanan_amount)), il s'agit d'une « caractéristique profonde » avec une profondeur de 1. Pour créer une caractéristique avec une profondeur de deux, nous pourrions empiler les primitives en prenant la valeur maximale des paiements mensuels moyens d'un client par prêt précédent (comme MAX(previous(MEAN(installments.payment)))). L'article original sur l'ingénierie automatisée des caractéristiques à l'aide de la synthèse profonde des caractéristiques vaut la peine d'être lu.

Pour réaliser la DFS dans featuretools, nous utilisons la fonction dfs en lui passant un entityset, l'entité_cible (où nous voulons créer les caractéristiques), les agg_primitives à utiliser, les trans_primitives à utiliser et la max_depth des caractéristiques. Ici, nous utiliserons les primitives d'agrégation et de transformation par défaut, une profondeur maximale de 2, et nous calculerons les primitives pour l'entité app. Comme ce processus est coûteux en calcul, nous pouvons exécuter la fonction en utilisant features_only = True pour ne renvoyer qu'une liste de caractéristiques et ne pas calculer les caractéristiques elles-mêmes. Cela peut être utile pour regarder les caractéristiques résultantes avant de commencer un calcul étendu.

**DFS with Default Primitive**

In [48]:
# Default primitives from featuretools
default_agg_primitives =  ["sum", "mean", "std", "max", "min", "count"]
default_trans_primitives =  ["day", "year", "month", "weekday"]

# DFS with specified primitives using updated parameter names
features, feature_names = ft.dfs(entityset=es, 
                                 target_dataframe_name='app_train', 
                                 trans_primitives=default_trans_primitives,
                                 agg_primitives=default_agg_primitives, 
                                 max_depth=2, 
                                 features_only=False)  # Set `features_only=True` if you only need feature definitions

print(f'{len(feature_names)} Total Features')

1553 Total Features


In [50]:
# Run DFS with the correct parameters
feature_matrix, feature_defs = ft.dfs(entityset=es, 
                                      target_dataframe_name='app_train',
                                      trans_primitives=default_trans_primitives,
                                      agg_primitives=default_agg_primitives, 
                                      max_depth=2, 
                                      features_only=False,  # Set True if only names are needed
                                      verbose=True)

# Display settings to view the dataframe properly
pd.options.display.max_columns = None  # Adjust as necessary
feature_matrix.head(10)

Built 1553 features
Elapsed: 00:01 | Progress: 100%|███████████████████████████████████████████████████████████████████████████████████████


TARGET NAME_CONTRACT_TYPE CODE_GENDER  FLAG_OWN_CAR  \
SK_ID_CURR                                                        
100002           1         Cash loans           M         False   
100003           0         Cash loans           F         False   
100004           0    Revolving loans           M          True   
100006           0         Cash loans           F         False   
100007           0         Cash loans           M         False   
100008           0         Cash loans           M         False   
100009           0         Cash loans           F          True   
100010           0         Cash loans           M          True   
100011           0         Cash loans           F         False   
100012           0    Revolving loans           M         False   

            FLAG_OWN_REALTY  CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  \
SK_ID_CURR                                                                
100002                 True             0          202500.0    406597.5   
100003                False             0          270000.0   1293502.5   
100004                 True             0           67500.0    135000.0   
100006                 True             0          135000.0    312682.5   
100007                 True             0          121500.0    513000.0   
100008                 True             0           99000.0    490495.5   
100009                 True             1          171000.0   1560726.0   
100010                 True             0          360000.0   1530000.0   
100011                 True             0          112500.0   1019610.0   
100012                 True             0          135000.0    405000.0   

            AMT_ANNUITY  AMT_GOODS_PRICE  NAME_TYPE_SUITE  \
SK_ID_CURR                                                  
100002          24700.5           351000    Unaccompanied   
100003          35698.5          1129500           Family   
100004           6750.0           135000    Unaccompanied   
100006          29686.5           297000    Unaccompanied   
100007          21865.5           513000    Unaccompanied   
100008          27517.5           454500  Spouse, partner   
100009          41301.0          1395000    Unaccompanied   
100010          42075.0          1530000    Unaccompanied   
100011          33826.5           913500         Children   
100012          20250.0           405000    Unaccompanied   

                NAME_INCOME_TYPE            NAME_EDUCATION_TYPE  \
SK_ID_CURR                                                        
100002                   Working  Secondary / secondary special   
100003             State servant               Higher education   
100004                   Working  Secondary / secondary special   
100006                   Working  Secondary / secondary special   
100007                   Working  Secondary / secondary special   
100008             State servant  Secondary / secondary special   
100009      Commercial associate               Higher education   
100010             State servant               Higher education   
100011                 Pensioner  Secondary / secondary special   
100012                   Working  Secondary / secondary special   

              NAME_FAMILY_STATUS  NAME_HOUSING_TYPE  \
SK_ID_CURR                                            
100002      Single / not married  House / apartment   
100003                   Married  House / apartment   
100004      Single / not married  House / apartment   
100006            Civil marriage  House / apartment   
100007      Single / not married  House / apartment   
100008                   Married  House / apartment   
100009                   Married  House / apartment   
100010                   Married  House / apartment   
100011                   Married  House / apartment   
100012      Single / not married  House / apartment   

            REGION_POPULATION_RELATIVE  DAYS_BIRTH  DAYS_EMPLOYED  \
SK_ID_CURR                            

In [51]:
feature_names[-20:]

[<Feature: STD(credit.previous.SELLERPLACE_AREA)>,
 <Feature: SUM(credit.previous.AMT_ANNUITY)>,
 <Feature: SUM(credit.previous.AMT_APPLICATION)>,
 <Feature: SUM(credit.previous.AMT_CREDIT)>,
 <Feature: SUM(credit.previous.AMT_DOWN_PAYMENT)>,
 <Feature: SUM(credit.previous.AMT_GOODS_PRICE)>,
 <Feature: SUM(credit.previous.CNT_PAYMENT)>,
 <Feature: SUM(credit.previous.DAYS_DECISION)>,
 <Feature: SUM(credit.previous.DAYS_FIRST_DRAWING)>,
 <Feature: SUM(credit.previous.DAYS_FIRST_DUE)>,
 <Feature: SUM(credit.previous.DAYS_LAST_DUE)>,
 <Feature: SUM(credit.previous.DAYS_LAST_DUE_1ST_VERSION)>,
 <Feature: SUM(credit.previous.DAYS_TERMINATION)>,
 <Feature: SUM(credit.previous.HOUR_APPR_PROCESS_START)>,
 <Feature: SUM(credit.previous.NFLAG_INSURED_ON_APPROVAL)>,
 <Feature: SUM(credit.previous.NFLAG_LAST_APPL_IN_DAY)>,
 <Feature: SUM(credit.previous.RATE_DOWN_PAYMENT)>,
 <Feature: SUM(credit.previous.RATE_INTEREST_PRIMARY)>,
 <Feature: SUM(credit.previous.RATE_INTEREST_PRIVILEGED)>,
 <Feature:

**DFS avec des primitives d'agrégation sélectionnées**
Avec featuretools, nous avons pu passer de 121 caractéristiques originales à près de 1553 en quelques lignes de code. Lorsque je faisais de l'ingénierie des caractéristiques à la main, il me fallait environ 12 heures pour créer un ensemble de données de taille comparable. Cependant, bien que nous obtenions beaucoup de caractéristiques dans featuretools, cet appel de fonction n'est pas très bien informé. Nous avons simplement utilisé les agrégations par défaut sans réfléchir à celles qui sont « importantes » pour le problème. Nous nous retrouvons avec un grand nombre de caractéristiques, mais elles ne sont probablement pas toutes pertinentes pour le problème. Un trop grand nombre de caractéristiques non pertinentes peut diminuer les performances en noyant les caractéristiques importantes (ce qui est lié à la malédiction de la dimensionnalité).

Le prochain appel que nous ferons spécifiera un ensemble plus restreint de caractéristiques. Nous n'utilisons toujours pas beaucoup de connaissances du domaine, mais cet ensemble de caractéristiques sera plus facile à gérer. L'étape suivante consiste à améliorer les caractéristiques que nous construisons réellement et à effectuer une sélection des caractéristiques.

In [53]:
# Specify the aggregation primitives
feature_matrix_spec, feature_names_spec = ft.dfs(entityset = es, 
                                                 target_dataframe_name = 'app_train',  
                                                 agg_primitives = ['sum', 'count', 'min', 'max', 'mean', 'mode'], 
                                                 max_depth = 2, 
                                                 features_only = False, 
                                                 verbose = True)

Built 1206 features
Elapsed: 00:01 | Progress: 100%|███████████████████████████████████████████████████████████████████████████████████████


Cela ne nous donne « que » 1206 caractéristiques 

In [54]:
pd.options.display.max_columns = 1000
feature_matrix_spec.head(10)

TARGET NAME_CONTRACT_TYPE CODE_GENDER  FLAG_OWN_CAR  \
SK_ID_CURR                                                        
100002           1         Cash loans           M         False   
100003           0         Cash loans           F         False   
100004           0    Revolving loans           M          True   
100006           0         Cash loans           F         False   
100007           0         Cash loans           M         False   
100008           0         Cash loans           M         False   
100009           0         Cash loans           F          True   
100010           0         Cash loans           M          True   
100011           0         Cash loans           F         False   
100012           0    Revolving loans           M         False   

            FLAG_OWN_REALTY  CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  \
SK_ID_CURR                                                                
100002                 True             0          202500.0    406597.5   
100003                False             0          270000.0   1293502.5   
100004                 True             0           67500.0    135000.0   
100006                 True             0          135000.0    312682.5   
100007                 True             0          121500.0    513000.0   
100008                 True             0           99000.0    490495.5   
100009                 True             1          171000.0   1560726.0   
100010                 True             0          360000.0   1530000.0   
100011                 True             0          112500.0   1019610.0   
100012                 True             0          135000.0    405000.0   

            AMT_ANNUITY  AMT_GOODS_PRICE  NAME_TYPE_SUITE  \
SK_ID_CURR                                                  
100002          24700.5           351000    Unaccompanied   
100003          35698.5          1129500           Family   
100004           6750.0           135000    Unaccompanied   
100006          29686.5           297000    Unaccompanied   
100007          21865.5           513000    Unaccompanied   
100008          27517.5           454500  Spouse, partner   
100009          41301.0          1395000    Unaccompanied   
100010          42075.0          1530000    Unaccompanied   
100011          33826.5           913500         Children   
100012          20250.0           405000    Unaccompanied   

                NAME_INCOME_TYPE            NAME_EDUCATION_TYPE  \
SK_ID_CURR                                                        
100002                   Working  Secondary / secondary special   
100003             State servant               Higher education   
100004                   Working  Secondary / secondary special   
100006                   Working  Secondary / secondary special   
100007                   Working  Secondary / secondary special   
100008             State servant  Secondary / secondary special   
100009      Commercial associate               Higher education   
100010             State servant               Higher education   
100011                 Pensioner  Secondary / secondary special   
100012                   Working  Secondary / secondary special   

              NAME_FAMILY_STATUS  NAME_HOUSING_TYPE  \
SK_ID_CURR                                            
100002      Single / not married  House / apartment   
100003                   Married  House / apartment   
100004      Single / not married  House / apartment   
100006            Civil marriage  House / apartment   
100007      Single / not married  House / apartment   
100008                   Married  House / apartment   
100009                   Married  House / apartment   
100010                   Married  House / apartment   
100011                   Married  House / apartment   
100012      Single / not married  House / apartment   

            REGION_POPULATION_RELATIVE  DAYS_BIRTH  DAYS_EMPLOYED  \
SK_ID_CURR                            

**Notes sur la mise en œuvre de base**
Ces appels ne représentent qu'une petite partie des capacités de featuretools. Nous n'avons pas spécifié les types de variables lors de la création d'entités, nous n'avons pas utilisé les variables de temps relatif et nous n'avons pas abordé les primitives personnalisées, les caractéristiques de départ ou les valeurs intéressantes ! Néanmoins, dans ce carnet, nous avons pu apprendre les bases fondamentales qui nous permettront d'utiliser plus efficacement l'outil au fur et à mesure que nous apprendrons comment il fonctionne. Voyons maintenant quelques-unes des fonctions que nous avons construites et les résultats de la modélisation.

**Résultats**
Pour déterminer si notre implémentation de base de featuretools a été utile, nous pouvons examiner plusieurs résultats :

- Les scores de validation croisée et les scores du classement public en utilisant plusieurs ensembles différents de caractéristiques.
- Corrélations : à la fois entre les caractéristiques et la cible, et entre les caractéristiques elles-mêmes.
- Feature Importance : déterminée par un modèle de gradient boosting 

**Expériences sur la performance des caractéristiques**
Afin de comparer un certain nombre d'ensembles de caractéristiques différents pour la tâche d'apprentissage automatique, j'ai mis en place plusieurs expériences. Afin d'isoler l'effet des caractéristiques, le même modèle a été utilisé pour tester un certain nombre d'ensembles de caractéristiques différents. Le modèle (qui peut être consulté en annexe) est un algorithme LightGBM de base utilisant la validation croisée 5 fois pour la formation et l'évaluation. Nous commençons par établir un ensemble de données de contrôle, puis nous effectuons une série d'expériences et présentons les résultats.

- Contrôle : utilisation des seules données de l'ensemble de données de l'application
- Test 1 : ingénierie manuelle des caractéristiques utilisant uniquement les données de l'application, du bureau et du bureau_balance
- Deuxième test : ingénierie manuelle des caractéristiques à l'aide de tous les ensembles de données
- Troisième test : caractéristiques par défaut de featuretools (dans la matrice feature_matrix)
- Test quatre : caractéristiques spécifiées par featuretools (dans feature_matrix_spec)
- Test cinq : caractéristiques spécifiées par featuretools combinées à l'ingénierie manuelle des caractéristiques

Le nombre de caractéristiques est indiqué après l'encodage à un coup, l'aire sous la courbe de la caractéristique d'exploitation du récepteur de validation (ROC AUC) est calculée en utilisant la validation croisée 5 fois, l'aire sous la courbe de la caractéristique d'exploitation du récepteur de test est tirée du classement public, et le temps passé à la conception est ma meilleure estimation du temps qu'il a fallu pour créer l'ensemble de données !

In [ ]:
# Training data
app_train = pd.read_csv('../input/application_train.csv')
print('Training data shape: ', app_train.shape)
app_train.head()

Le train comporte 307511 observations (chacune étant un prêt distinct) et 122 variables, y compris la target

In [ ]:
# Testing data features
app_test = pd.read_csv('../input/application_test.csv')
print('Testing data shape: ', app_test.shape)
app_test.head()

Le test comporte bcp moins d'observations (48744) et ne comporte pas de  TARGET. Nous n'en tiendrons pas compte dans un 1er temps

## 1.3 - Exploratory Data Analysis

### 1.3.1 -  Distribution de la target

In [ ]:
app_train['TARGET'].value_counts()

In [ ]:
app_train['TARGET'].value_counts(normalize=True)

In [ ]:
app_train['TARGET'].plot.hist();

Ces informations montrent qu'il s'agit d'un problème de classe déséquilibrée :  http://www.chioka.in/class-imbalance-problem/. 

Il y a beaucoup plus de prêts remboursés à temps que de prêts non remboursés. Lorsque nous utiliserons des modèles d'apprentissage automatique plus sophistiqués, nous pourrons pondérer les classes : http://xgboost.readthedocs.io/en/latest/parameter.html en fonction de leur représentation dans les données afin de refléter ce déséquilibre :

- scale_pos_weight : C'est l'un des hyperparamètres les plus importants pour les ensembles de données déséquilibrés dans les problèmes de classification binaire. Ce paramètre donne plus de poids à la classe minoritaire pendant l'entraînement, aidant le modèle à accorder une attention appropriée à cette classe.
- max_delta_step : Cet hyperparamètre aide à rendre la mise à jour du gradient plus conservatrice, souvent utile dans les ensembles de données très déséquilibrés. Il est généralement utilisé en conjonction avec scale_pos_weight pour améliorer la performance du modèle.

Paramètres relatifs à l'échantillonnage :

- subsample : Le pourcentage d'échantillons utilisés pour chaque arbre. Réduire cette valeur peut aider à éviter le surajustement.
- colsample_bytree, colsample_bylevel, colsample_bynode : Ces paramètres contrôlent le sous-échantillonnage des caractéristiques à différents niveaux de la construction des arbres. Ils peuvent être utiles pour gérer les surajustements.

En général, ajuster scale_pos_weight est un bon point de départ pour traiter des ensembles de données déséquilibrés. Il est également recommandé de faire une validation croisée pour déterminer les valeurs optimales de ces hyperparamètres pour votre ensemble de données spécifique.

### 1.3.2 - Missing Values

In [ ]:
# Function to calculate missing values by column# Funct 
def missing_values_table(df, print_info = False):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        if print_info:
            # Print some summary information
            print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
                "There are " + str(mis_val_table_ren_columns.shape[0]) +
                  " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [ ]:
# Missing values statistics
missing_values = missing_values_table(app_train)
missing_values.head(20)

Lorsqu'il s'agira de construire nos modèles d'apprentissage automatique, nous devrons imputer ces valeurs manquantes. Dans un travail ultérieur, nous utiliserons des modèles tels que XGBoost qui peuvent traiter les valeurs manquantes sans nécessiter d'imputation : https://stats.stackexchange.com/questions/235489/xgboost-can-handle-missing-data-in-the-forecasting-phase.
Une autre option consisterait à supprimer les colonnes présentant un pourcentage élevé de valeurs manquantes, bien qu'il soit impossible de savoir à l'avance si ces colonnes seront utiles à notre modèle. 
Par conséquent, nous conserverons toutes les colonnes pour l'instant.

### 1.3.3 - Column Types

In [ ]:
# Number of each type of column
app_train.dtypes.value_counts()

In [ ]:
# Number of unique classes in each object column
app_train.select_dtypes('object').apply(pd.Series.nunique, axis = 0)

La plupart des variables catégorielles ont un nombre relativement faible d'entrées uniques.

### 1.3.4 - Anomalies

Les anomalies dans les données sont un problème que nous devons toujours surveiller lorsque nous faisons de l'AED. Celles-ci peuvent être dues à des chiffres mal saisis, à des erreurs dans les mesures ou à des mesures valables mais extrêmes. L'une des façons de détecter les anomalies de manière quantitative est d'examiner les statistiques d'une colonne à l'aide de la méthode `describe`. 

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
app_train.describe()

Les chiffres de la colonne `DAYS_BIRTH` sont négatifs car ils sont enregistrés par rapport à la demande de prêt en cours. Pour voir ces statistiques en années, nous pouvons multiplier par -1 et diviser par le nombre de jours dans une année :

In [ ]:
(app_train['DAYS_BIRTH'] / -365).describe()

Ces âges semblent raisonnables. Il n'y a pas de valeurs aberrantes pour l'âge, que ce soit à l'extrémité supérieure ou inférieure. Qu'en est-il des jours d'emploi ? 

In [ ]:
app_train['DAYS_EMPLOYED'].describe()

Cela ne semble pas correct ! La valeur maximale (en plus d'être positive) est d'environ 1000 ans !

In [ ]:
app_train['DAYS_EMPLOYED'].plot.hist(title = 'Days Employment Histogram');
plt.xlabel('Days Employment');

Par simple curiosité, nous allons regrouper les clients présentant des anomalies et voir s'ils ont tendance à avoir des taux de défaillance plus élevés ou plus faibles que le reste des clients.

In [ ]:
anom = app_train[app_train['DAYS_EMPLOYED'] == 365243]
non_anom = app_train[app_train['DAYS_EMPLOYED'] != 365243]
print('The non-anomalies default on %0.2f%% of loans' % (100 * non_anom['TARGET'].mean()))
print('The anomalies default on %0.2f%% of loans' % (100 * anom['TARGET'].mean()))
print('There are %d anomalous days of employment' % len(anom))

Il s'avère que les anomalies ont un taux de défaillance plus faible. 

Le traitement des anomalies dépend de la situation exacte et il n'existe pas de règles fixes. L'une des approches les plus sûres consiste à définir les anomalies comme des valeurs manquantes et à les compléter (à l'aide de l'imputation) avant l'apprentissage automatique. Dans ce cas, comme toutes les anomalies ont exactement la même valeur, nous voulons les compléter avec la même valeur au cas où tous ces prêts auraient quelque chose en commun. Les valeurs anormales semblent avoir une certaine importance, c'est pourquoi nous voulons indiquer au modèle d'apprentissage automatique si nous avons effectivement renseigné ces valeurs. Comme solution, nous allons remplir les valeurs anormales avec un nombre (`np.nan`) et ensuite créer une nouvelle colonne booléenne indiquant si la valeur était anormale ou non.



In [ ]:
# Create an anomalous flag column
app_train['DAYS_EMPLOYED_ANOM'] = app_train["DAYS_EMPLOYED"] == 365243

# Replace the anomalous values with nan
app_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)

app_train['DAYS_EMPLOYED'].plot.hist(title = 'Days Employment Histogram');
plt.xlabel('Days Employment');

La distribution semble beaucoup plus conforme à ce que nous attendons, et nous avons également créé une nouvelle colonne pour indiquer au modèle que ces valeurs étaient à l'origine anormales (parce que nous devrons remplir les nans avec une valeur, probablement la médiane de la colonne). Les autres colonnes avec `DAYS` dans le cadre de données semblent être à peu près ce que nous attendons, sans valeurs aberrantes évidentes. 

In [ ]:
app_train['AMT_ANNUITY'].plot.hist(title = 'Annuity Histogram');
plt.xlabel('Annuity');

### 1.3.5 - Correlations

Une façon d'essayer de comprendre les données est de rechercher des corrélations entre les caractéristiques et la cible. Nous pouvons calculer le coefficient de corrélation de Pearson entre chaque variable et la cible à l'aide de la méthode .corr dataframe.

Le coefficient de corrélation n'est pas la meilleure méthode pour représenter la « pertinence » d'une caractéristique, mais il nous donne une idée des relations possibles entre les données. Voici quelques interprétations générales de la valeur absolue du coefficient de corrélation :

* .00-.19 “very weak”
*  .20-.39 “weak”
*  .40-.59 “moderate”
*  .60-.79 “strong”
* .80-1.0 “very strong”


In [ ]:
numeric_columns=app_train.select_dtypes(include=np.number).columns.tolist()
numeric_columns

# Matrice de corrélation
correlations = app_train[numeric_columns].corr()['TARGET'].sort_values()

# Display correlations
print('Most Positive Correlations:\n', correlations.tail(15))
print('\nMost Negative Correlations:\n', correlations.head(15))

La corrélation `DAYS_BIRTH` est la plus positive. En regardant la documentation, `DAYS_BIRTH` est l'âge en jours du client au moment du prêt en jours négatifs (pour une raison quelconque !). La corrélation est positive, mais la valeur de cette caractéristique est en fait négative, ce qui signifie que plus le client vieillit, moins il est susceptible de ne pas rembourser son prêt (c'est-à-dire la cible == 0). C'est un peu déroutant, nous prendrons donc la valeur absolue de la caractéristique et la corrélation sera négative.

#### 1.3.5.1 - Effect of Age on Repayment

In [ ]:
# Find the correlation of the positive days since birth and target
app_train['DAYS_BIRTH'] = abs(app_train['DAYS_BIRTH'])
app_train['DAYS_BIRTH'].corr(app_train['TARGET'])

Plus le client est âgé, plus il existe une relation linéaire négative avec la cible, ce qui signifie que plus les clients sont âgés, plus ils ont tendance à rembourser leurs prêts à temps. 

Commençons par examiner cette variable. Tout d'abord, nous pouvons faire un histogramme de l'âge. L'axe des x sera exprimé en années pour rendre le graphique un peu plus compréhensible.

In [ ]:
# Set the style of plots
plt.style.use('fivethirtyeight')

# Plot the distribution of ages in years
plt.hist(app_train['DAYS_BIRTH'] / 365, edgecolor = 'k', bins = 25)
plt.title('Age of Client'); plt.xlabel('Age (years)'); plt.ylabel('Count');

En soi, la distribution de l'âge ne nous apprend pas grand-chose, si ce n'est qu'il n'y a pas de valeurs aberrantes puisque tous les âges sont raisonnables.
Pour visualiser l'effet de l'âge sur la cible, nous ferons ensuite un graphique d'estimation de la densité du noyau (KDE) coloré par la valeur de la cible. Un graphique d'estimation de la densité du noyau montre la distribution d'une seule variable et peut être considéré comme un histogramme lissé (il est créé en calculant un noyau, généralement une gaussienne, à chaque point de données, puis en faisant la moyenne de tous les noyaux individuels pour développer une seule courbe lisse). Nous utiliserons le `kdeplot` de seaborn pour ce graphique.

In [ ]:
plt.figure(figsize = (8, 5))

# KDE plot of loans that were repaid on time
sns.kdeplot(app_train.loc[app_train['TARGET'] == 0, 'DAYS_BIRTH'] / 365, label = 'target == 0')

# KDE plot of loans which were not repaid on time
sns.kdeplot(app_train.loc[app_train['TARGET'] == 1, 'DAYS_BIRTH'] / 365, label = 'target == 1')

# Labeling of plot
plt.xlabel('Age (years)'); plt.ylabel('Density'); plt.title('Distribution of Ages');

La courbe de la Target == 1 penche vers l'extrémité la plus jeune de la fourchette. Bien qu'il ne s'agisse pas d'une corrélation significative (coefficient de corrélation de -0,07), cette variable sera probablement utile dans un modèle d'apprentissage automatique car elle affecte la cible. Examinons cette relation sous un autre angle : le taux moyen de non-remboursement des prêts par tranche d'âge. 

Pour créer ce graphique, nous commençons par « découper » la catégorie d'âge en tranches de 5 ans chacune. Ensuite, pour chaque tranche, nous calculons la valeur moyenne de la cible, qui nous indique le ratio de prêts non remboursés dans chaque catégorie d'âge.

In [ ]:
# Age information into a separate dataframe
age_data = app_train[['TARGET', 'DAYS_BIRTH']]
age_data['YEARS_BIRTH'] = age_data['DAYS_BIRTH'] / 365

# Bin the age data
age_data['YEARS_BINNED'] = pd.cut(age_data['YEARS_BIRTH'], bins = np.linspace(20, 70, num = 11))
age_data.head(10)

In [ ]:
# Group by the bin and calculate averages
age_groups  = age_data.groupby('YEARS_BINNED').mean()
age_groups

In [ ]:
plt.figure(figsize = (5, 5))

# Graph the age bins and the average of the target as a bar plot
plt.bar(age_groups.index.astype(str), 100 * age_groups['TARGET'])

# Plot labeling
plt.xticks(rotation = 75); plt.xlabel('Age Group (years)'); plt.ylabel('Failure to Repay (%)')
plt.title('Failure to Repay by Age Group');

La tendance est claire : les jeunes demandeurs sont plus susceptibles de ne pas rembourser le prêt ! Le taux de non-remboursement est supérieur à 10 % pour les trois groupes d'âge les plus jeunes et inférieur à 5 % pour le groupe le plus âgé.

Il s'agit d'une information qui pourrait être directement utilisée par la banque : étant donné que les jeunes clients sont moins susceptibles de rembourser le prêt, il conviendrait peut-être de leur fournir davantage de conseils ou d'astuces en matière de planification financière. Cela ne signifie pas que la banque doive discriminer les jeunes clients, mais il serait judicieux de prendre des mesures de précaution pour les aider à rembourser à temps.

#### 1.3.5.2 - Exterior Sources

Les 3 variables ayant les corrélations négatives les plus fortes avec la cible sont `EXT_SOURCE_1`, `EXT_SOURCE_2`, et `EXT_SOURCE_3`.
Selon la documentation, ces caractéristiques représentent un « score normalisé de la source de données externe ». Je ne suis pas sûr de ce que cela signifie exactement, mais il pourrait s'agir d'une sorte d'évaluation de crédit cumulative réalisée à partir de nombreuses sources de données. 

Examinons ces variables.

Tout d'abord, nous pouvons montrer les corrélations des caractéristiques `EXT_SOURCE` avec la cible et entre elles.

In [ ]:
# Extract the EXT_SOURCE variables and show correlations
ext_data = app_train[['TARGET', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']]
ext_data_corrs = ext_data.corr()
ext_data_corrs

In [ ]:
# plt.figure(figsize = (8, 6))

# # Heatmap of correlations
# sns.heatmap(ext_data_corrs, cmap = plt.cm.RdYlBu_r, vmin = -0.25, annot = True, vmax = 0.6)
# plt.title('Correlation Heatmap');

Les trois caractéristiques EXT_SOURCE ont des corrélations négatives avec la cible, ce qui indique que plus la valeur de l'EXT_SOURCE augmente, plus le client est susceptible de rembourser le prêt. Nous pouvons également constater que DAYS_BIRTH est positivement corrélé avec EXT_SOURCE_1, ce qui indique que l'âge du client est peut-être l'un des facteurs de ce score.

Nous pouvons ensuite examiner la distribution de chacune de ces caractéristiques en fonction de la valeur de la cible. Cela nous permettra de visualiser l'effet de cette variable sur la cible.

In [ ]:
plt.figure(figsize = (15, 8))

# iterate through the sources
for i, source in enumerate(['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']):
    
    # create a new subplot for each source
    plt.subplot(3, 1, i + 1)
    # plot repaid loans
    sns.kdeplot(app_train.loc[app_train['TARGET'] == 0, source], label = 'target == 0')
    # plot loans that were not repaid
    sns.kdeplot(app_train.loc[app_train['TARGET'] == 1, source], label = 'target == 1')
    
    # Label the plots
    plt.title('Distribution of %s by Target Value' % source)
    plt.xlabel('%s' % source); plt.ylabel('Density');
    
plt.tight_layout(h_pad = 2.5)
    

`EXT_SOURCE_3` affiche la plus grande différence entre les valeurs de la cible. Nous pouvons clairement voir que cette caractéristique a une certaine relation avec la probabilité qu'un demandeur rembourse un prêt. La relation n'est pas très forte (en fait, elles sont toutes considérées comme très faibles, mais ces variables seront toujours utiles pour un modèle d'apprentissage automatique permettant de prédire si un demandeur remboursera ou non un prêt dans les délais.

### 1.3.6 - Pairs Plot

En guise de dernier graphique exploratoire, nous pouvons faire un pairplot des variables `EXT_SOURCE` et de la variable `DAYS_BIRTH`. Le Pairplot (https://towardsdatascience.com/visualizing-data-with-pair-plots-in-python-f228cf529166) est un excellent outil d'exploration car il nous permet de voir les relations entre plusieurs paires de variables ainsi que les distributions de variables uniques. 

In [ ]:
# # Copy the data for plotting
# plot_data = ext_data.drop(columns = ['DAYS_BIRTH']).copy()

# # Add in the age of the client in years
# plot_data['YEARS_BIRTH'] = age_data['YEARS_BIRTH']

# # Drop na values and limit to first 100000 rows
# plot_data = plot_data.dropna().loc[:100000, :]

# # Function to calculate correlation coefficient between two columns
# def corr_func(x, y, **kwargs):
#     r = np.corrcoef(x, y)[0][1]
#     ax = plt.gca()
#     ax.annotate("r = {:.2f}".format(r),
#                 xy=(.2, .8), xycoords=ax.transAxes,
#                 size = 20)

# # Create the pairgrid object
# grid = sns.PairGrid(data = plot_data, height = 3, diag_sharey=False,
#                     hue = 'TARGET', 
#                     vars = [x for x in list(plot_data.columns) if x != 'TARGET'])

# # Upper is a scatter plot
# grid.map_upper(plt.scatter, alpha = 0.2)

# # Diagonal is a histogram
# grid.map_diag(sns.kdeplot)

# # Bottom is density plot
# grid.map_lower(sns.kdeplot, cmap = plt.cm.OrRd_r);

# plt.suptitle('Ext Source and Age Features Pairs Plot', size = 32, y = 1.05);

Dans ce graphique, le rouge indique les prêts non remboursés et le bleu les prêts remboursés. Nous pouvons voir les différentes relations au sein des données. Il semble y avoir une relation linéaire positive modérée entre EXT_SOURCE_1 et DAYS_BIRTH (ou YEARS_BIRTH), ce qui indique que cette caractéristique peut prendre en compte l'âge du client.

## 1.4 - Feature Engineering

Le Feature Engineering a un meilleur retour sur investissement que la construction de modèles et l'ajustement des hyperparamètres. 

Feature Engineering se réfère à un processus général et peut impliquer à la fois la construction de caractéristiques : l'ajout de nouvelles caractéristiques à partir des données existantes, et la sélection de caractéristiques : le choix des caractéristiques les plus importantes ou d'autres méthodes de réduction de la dimensionnalité. Il existe de nombreuses techniques permettant de créer et de sélectionner des caractéristiques.


Nous pouvons créer quelques caractéristiques qui tentent de capturer ce que nous pensons être important pour savoir si un client va manquer à ses obligations de remboursement d'un prêt. Ici, je vais utiliser cinq caractéristiques qui ont été inspirées par ce script (https://www.kaggle.com/jsaguiar/updated-0-792-lb-lightgbm-with-simple-features) d'Aguiar :

* `CREDIT_INCOME_PERCENT` : le pourcentage du montant du crédit par rapport au revenu du client.
* `ANNUITY_INCOME_PERCENT` : le pourcentage de l'annuité du prêt par rapport au revenu du client
* `CREDIT_TERM` : la durée du paiement en mois (puisque l'annuité est le montant mensuel dû).
* `DAYS_EMPLOYED_PERCENT` : le pourcentage des jours de travail par rapport à l'âge du client.


In [ ]:
app_train_domain = app_train.copy()

app_train_domain['CREDIT_INCOME_PERCENT'] = app_train_domain['AMT_CREDIT'] / app_train_domain['AMT_INCOME_TOTAL']
app_train_domain['ANNUITY_INCOME_PERCENT'] = app_train_domain['AMT_ANNUITY'] / app_train_domain['AMT_INCOME_TOTAL']
app_train_domain['CREDIT_TERM'] = app_train_domain['AMT_ANNUITY'] / app_train_domain['AMT_CREDIT']
app_train_domain['DAYS_EMPLOYED_PERCENT'] = app_train_domain['DAYS_EMPLOYED'] / app_train_domain['DAYS_BIRTH']

Nous devrions explorer ces variables de __connaissance du domaine__ visuellement dans un graphique. Pour toutes ces variables, nous ferons le même graphique KDE coloré par la valeur de la `TARGET`.

In [ ]:
plt.figure(figsize = (12, 20))
# iterate through the new features
for i, feature in enumerate(['CREDIT_INCOME_PERCENT', 'ANNUITY_INCOME_PERCENT', 'CREDIT_TERM', 'DAYS_EMPLOYED_PERCENT']):
    
    # create a new subplot for each source
    plt.subplot(4, 1, i + 1)
    # plot repaid loans
    sns.kdeplot(app_train_domain.loc[app_train_domain['TARGET'] == 0, feature], label = 'target == 0')
    # plot loans that were not repaid
    sns.kdeplot(app_train_domain.loc[app_train_domain['TARGET'] == 1, feature], label = 'target == 1')
    
    # Label the plots
    plt.title('Distribution of %s by Target Value' % feature)
    plt.xlabel('%s' % feature); plt.ylabel('Density');
    
plt.tight_layout(h_pad = 2.5)

Il est difficile de dire à l'avance si ces nouvelles fonctionnalités seront utiles. Le seul moyen d'en être sûr est de les essayer ! 

# 2 - Modélisation

In [ ]:
# démarrage du tracking
mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

### 2.1 - Split train en train/test

Je garde une copie du df intact avec les id clients et la target

In [ ]:
app_train_origin=app_train_domain.copy()

In [ ]:
X = app_train_domain.drop(['SK_ID_CURR','TARGET'], axis=1)  
y = app_train_domain['TARGET']  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### 2.2 - Encoding Variables catégorielles

Un modèle d'apprentissage automatique ne peut pas traiter les variables catégorielles (à l'exception de certains modèles tels que LightGBM : http://lightgbm.readthedocs.io/en/latest/Features.html. 

Nous devons donc trouver un moyen d'encoder (représenter) ces variables sous forme de nombres avant de les transmettre au modèle. Il y a deux façons principales d'effectuer ce processus :
- Label encoding: attribue un nombre entier à chaque catégorie unique d'une variable catégorielle.
- One-hot encoding: créé une nouvelle colonne pour chaque catégorie unique dans une variable catégorielle. Chaque observation reçoit un 1 dans la colonne de la catégorie correspondante et un 0 dans toutes les autres nouvelles colonnes.

Le problème du Label encoding est qu'il donne aux catégories un ordre arbitraire. La valeur attribuée à chacune des catégories est aléatoire et ne reflète aucun aspect inhérent à la catégorie. Dans l'exemple ci-dessus, le programmeur reçoit un 4 et le scientifique des données un 1, mais si nous refaisions le même processus, les étiquettes pourraient être inversées ou complètement différentes. L'affectation réelle des nombres entiers est arbitraire. Par conséquent, lorsque nous procédons à l'encodage des étiquettes, le modèle peut utiliser la valeur relative de la caractéristique (par exemple, programmeur = 4 et scientifique = 1) pour attribuer des poids, ce qui n'est pas ce que nous voulons. Si nous n'avons que deux valeurs uniques pour une variable catégorielle (telle que Homme/Femme), le Label Encoding convient, mais pour plus de deux catégories uniques, le One Hot Encoding est l'option la plus sûre.


Le seul inconvénient du One Hot Encoding est que le nombre de caractéristiques (dimensions des données) peut exploser avec des variables catégorielles comportant de nombreuses catégories. Pour y remédier, nous pouvons effectuer un OHE suivi d'une ACP ou d'autres méthodes de réduction de la dimensionnalité afin de réduire le nombre de dimensions tout en essayant de préserver l'information. 


Dans ce NB, nous utiliserons le Label Encoding pour toutes les variables catégorielles comportant seulement deux catégories et le OHE pour toutes les variables catégorielles comportant plus de deux catégories. Il se peut que ce processus doive être modifié au fur et à mesure que nous avançons dans le projet, mais pour l'instant, nous allons voir où cela nous mène. Nous n'utiliserons pas non plus de réduction de la dimensionnalité dans ce carnet, mais nous l'explorerons dans des itérations futures.

In [ ]:
# Create a label encoder object
le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in app_train_domain:
    if app_train_domain[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(app_train_domain[col].unique())) <= 2:
            # Train on the training data
            le.fit(app_train_domain[col])
            # Transform  training, val and testing data
            X_train[col] = le.transform(X_train[col])
            X_test[col] = le.transform(X_test[col])
                        
            # Keep track of how many columns were label encoded
            le_count += 1
            
print('%d columns were label encoded.' % le_count)

In [ ]:
# one-hot encoding of categorical variables
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

print('Training Features shape: ', X_train.shape)
print('Test Features shape: ', X_test.shape)


Il doit y avoir les mêmes colonnes dans le train et dans le test. Si le OHE a créé plus de colonnes dans le train que dans le test (parce qu'il y avait des variables catégorielles dont les catégories n'étaient pas représentées dans les données de test), alors il faut supprimer les colonnes des données d'apprentissage qui ne se trouvent pas dans les données de test. Pour ça, nous devons « aligner » les cadres de données. 

In [ ]:
# # Align the training and testing data, keep only columns present in both dataframes
# X_train, X_test = X_train.align(X_test, join = 'inner', axis = 1)

# print('Training Features shape: ', X_train.shape)
# print('Val Features shape: ', X_test.shape)


Rmq : Lorsqu'on utilise la méthode align() avec l'option join='inner' pour aligner deux DataFrames pandas sur l'axe des colonnes, toutes les colonnes qui ne sont pas communes aux deux DataFrames sont supprimées du résultat final. Cela signifie que toute information contenue dans ces colonnes supprimées est perdue dans les DataFrames retournés par la méthode.

## 2.3 - Imputation MV et normalisation avec un min max scaler

Récupération des noms des colonnes avant imputation des MV et Standard Scaling pour extraire la feature importance après modélisation

In [ ]:
features = list(X_train.columns)
features

In [ ]:
# sauvegarde d'un X_test avant imputation des MV et standardscaling
X_test_save=X_test.copy()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Median imputation of missing values
imputer = SimpleImputer(strategy = 'median')

# Scale each feature to 0-1
scaler = MinMaxScaler(feature_range = (0, 1))

# Fit on the training data
imputer.fit(X_train)

# Transform both training and testing data
X_train = imputer.transform(X_train)
X_test = imputer.transform(X_test)

# Repeat with the scaler
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

print('Training data shape: ', X_train.shape)
print('Testing data shape: ', X_test.shape)


## 2.4 - Baseline = Dummy Classifier

Pour obtenir une baseline :
- nous utiliserons toutes les caractéristiques après avoir encodé les variables catégorielles.
- Nous allons prétraiter les données en remplissant les valeurs manquantes (imputation)
- et normaliser les caractéristiques.


### 2.4.1 - Entrainement et prédictions

Pour avoir une référence et, comme le jeu de données est déséquilibré, on va tester un modèle naïf avec la stratégie 'stratified', qui va générer des prédictions en respectant la distribution des classes de l'ensemble d'entrainement

In [ ]:
dummy_clf = DummyClassifier(strategy='stratified', random_state=42)

# Entraînement du Dummy Classifier
start_time=time.time()

dummy_clf.fit(X_train, y_train)

end_time = time.time()
training_execution_time = round(end_time - start_time, 2)
print(f"Le temps d'entrainement est de {training_execution_time} secondes.")

Maintenant que le modèle a été entraîné, nous pouvons l'utiliser pour faire des prédictions. Nous voulons prédire les probabilités de ne pas rembourser un prêt, nous utilisons donc la méthode `predict.proba` du modèle. Celle-ci renvoie un tableau m x 2 où m est le nombre d'observations. La première colonne est la probabilité que la cible soit 0 et la deuxième colonne est la probabilité que la cible soit 1 (donc pour une seule ligne, la somme des deux colonnes doit être égale à 1). Nous voulons connaître la probabilité que le prêt ne soit pas remboursé, nous sélectionnerons donc la deuxième colonne.

Le code suivant effectue les prédictions et sélectionne la colonne correcte.

In [ ]:
# Prédictions 
start_time=time.time()

y_pred_dummy_train = dummy_clf.predict(X_train)
dummy_pred_train = dummy_clf.predict_proba(X_train)[:, 1]

end_time = time.time()
predicting_execution_time = round(end_time - start_time, 2)
print(f"Le temps de prédiction sur le train est de {predicting_execution_time} secondes.")

y_pred_dummy_test = dummy_clf.predict(X_test)
dummy_pred_test = dummy_clf.predict_proba(X_test)[:, 1]


### 2.4.2 - Evaluation

#### 2.4.2.1 - Confusion matrix et Classification Reports

In [ ]:
pd.crosstab(y_test, y_pred_dummy_test, rownames=['Classes réelles'], colnames=['Classes prédites'])

In [ ]:
print(classification_report(y_test, y_pred_dummy_test))

#### 2.4.2.2 - AUC du train et du test

In [ ]:
# Calculer l'AUC sur le train
auc_train = round(roc_auc_score(y_train, dummy_pred_train),2)
print("Train AUC:", auc_train)

In [ ]:
# Calculer l'AUC sur le test
auc_test = round(roc_auc_score(y_test, dummy_pred_test),2)
print("Test AUC:", auc_test)

Avec ce modèle, on a 50% de chance que, en prenant 2 individus aléatoirement, l'individu le mieux scoré ait une target = 1.

### 2.4.3 - Elaboration du score métier

#### 2.4.3.1 - Définition de la matrice de coûts

- Faux Négatif (FN) : un mauvais client est prédit comme bon. Fixons ce coût à 10.
- Faux Positif (FP) : un bon client est prédit comme mauvais. Fixons ce coût à 1.

#### 2.4.3.2 - Calcul du coût total des erreurs

Pour chaque seuil de probabilité p utilisé pour classifier les observations en positives (clients défaillants) ou négatives (bons clients), on calcule le coût total des erreurs comme suit :
- on va calculer le nombre de FN et FP pour le seuil p
- on va calculer le coût total : coût total (p) = coût FN x Nbre FN + Coût FP x Nbre FP

In [ ]:
# fonction qui va établir un seuil pour convertir les probabiliiéts en étiquettes de classe binaire. 
# à partir de la matrice de confusion, on va calculer le coût

def calculate_cost_threshold(y_true, probas, threshold, cost_fn, cost_fp):
    y_pred = (probas >= threshold).astype(int) # transforme les probas en prédictions binaires en utilisant un seuil. 
                                                # Les probabilités >= au seuil sont marquées comme 1 (positives),
                                                # les autres comme 0 (négatives).

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel() # .ravel() convertit la matrice 2x2 en un tableau à une dimension [tn, fp, fn, tp]
                                                            # permet une extraction facile de chaque valeur
    return cost_fn * fn + cost_fp * fp

#### 2.4.3.3 - Optimisation du seuil

Il va s'agir d'utiliser une méthode pour trouver le seuil p qui minimise le Cout total.  
Cette étape peut impliquer :
- Analyse de la courbe des coûts : Tracage du coût total en fonction de différents seuils pour identifier visuellement le minimum.
- Recherche algorithmique :  Utilisation de techniques comme la recherche exhaustive dans un intervalle de seuils ou des méthodes plus sophistiquées comme l'optimisation numérique.

In [ ]:
thresholds = np.linspace(0, 1, 100) # génére 100 valeurs de seuil uniformément réparties entre 0 et 1

costs = [calculate_cost_threshold(y_test, dummy_pred_test, thr, cost_fn=10, cost_fp=1) for thr in thresholds]


In [ ]:
# on chrehce le seuil qui minimise le coût
optimal_threshold = round(thresholds[np.argmin(costs)],2) # détermine le seuil optimal en trouvant l'indice du coût minimal 
                                                        # dans la liste costs à l'aide de np.argmin(), 
                                                        # puis en récupérant la valeur correspondante dans thresholds. La valeur est arrondie à 2 décimales pour plus de lisibilité.
print(f"Optimal Threshold: {optimal_threshold}")

### 2.4.4 - Analyse des résultats du test au seuil optimal

#### 2.4.4.1 - Confusion matrix et Classification Reports au seuil optimal

In [ ]:
y_pred_optimal = (dummy_pred_test >= optimal_threshold).astype(int)

Rmq : (dummy_pred_test >= optimal_threshold) effectue une comparaison élément par élément entre les probabilités prédites et le seuil optimal. Cela crée un tableau de booléens où chaque élément est True si la probabilité prédite est supérieure ou égale au seuil optimal, sinon False.
.astype(int) est utilisé pour convertir les valeurs booléennes (True/False) en entiers (1/0). Ainsi, les valeurs supérieures ou égales au seuil optimal seront converties en 1 et les autres en 0.
Le résultat final, y_pred_optimal, est un tableau de prédictions binaires où chaque élément est 1 si la probabilité prédite correspondante est supérieure ou égale au seuil optimal, sinon 0.

In [ ]:
pd.crosstab(y_test, y_pred_optimal, rownames=['Classes réelles'], colnames=['Classes prédites'])

In [ ]:
print(classification_report(y_test, y_pred_optimal))

#### 2.4.4.2 - recall et f1 score de la classe 1 au seuil optimal

In [ ]:
report = classification_report(y_test, y_pred_optimal, output_dict=True) # output_dict=True indique à la fonction classification_report 
                                                    # de renvoyer le rapport sous forme de dictionnaire Python plutôt que sous forme de chaîne de caractères.

In [ ]:
report

In [ ]:
recall_1_test = round(report['1']['recall'],2)
print(f"Recall for class '1': {recall_1_test}")

In [ ]:
f1_score_1_test = round(report['1']['f1-score'],2)
print(f"F1 Score for class '1': {f1_score_1_test}")

#### 2.4.4.3 - Accuracy du test au seuil optimal

In [ ]:
accuracy_test = round(accuracy_score(y_test, y_pred_optimal),2)
print("Accuracy du test:", accuracy_test)

### 2.4.5 - Enregistrement du modèle dans MLFlow

In [ ]:
# # Création d'une nouvelle expérience MLflow
# mlflow.set_experiment("MLflow Credit_Scoring - Projet_7")

# # Commencer une session MLflow
# with mlflow.start_run():
#     # Log les hyperparameters que j'ai déclarés au dessus
#     # mlflow.log_params(params)

#     # Log la métrique de perte que j'ai déclarée au dessus
    
#     mlflow.log_metric("optimal_threshold", optimal_threshold)

#     mlflow.log_metric("recall_1_test", recall_1_test)
#     mlflow.log_metric("f1_score_1_test", f1_score_1_test)
#     mlflow.log_metric("accuracy du test", accuracy_test)
    
#     mlflow.log_metric("auc_train", auc_train)
#     mlflow.log_metric("auc_test", auc_test)
    
#     mlflow.log_metric("temps_fit", training_execution_time)
#     mlflow.log_metric("temps_predict", predicting_execution_time)

    
    

#     # Définir un tag pour se rappeler l'objet de cette session
#     mlflow.set_tag("Training Info", "Dummy model _ Methode Stratified ")

#     # Infer the model signature
#     signature = infer_signature(X_train, dummy_clf.predict(X_train)) # infer_signature génère automatiquement une "signature" 
#                                                               # qui décrit les entrées et les sorties du modèle. 
#                                                               # Cela inclut les types de données et les formats attendus par le modèle, 
#                                                               # facilitant ainsi l'intégration et la réutilisation du modèle 
#                                                               # dans différents environnements.

    
#     # Log le model
#     model_info = mlflow.sklearn.log_model(
#         sk_model=dummy_clf, # le modèle
#         artifact_path="dummy_model", # le chemin où le modèle sera enregistré
#         signature=signature, # la signature du modèle
#         input_example=X_train, # un exemple d'input pour montrer comment invoquer le modèle
#         registered_model_name="scoring-credit-dummy", # nom sous lequel le modèle est enregistré dans le registre de modèles MLflow
#     )


## 2.2 - Régression logistique 

### 2.2.1 - Définition des fonctions de scoring

In [ ]:
def custom_f1(y_true, y_pred_proba):
    thresholds = np.linspace(0, 1, 100)
    best_threshold = 0.5
    best_f1 = 0
    best_cost = float('inf')
    
    for threshold in thresholds:
        y_pred = (y_pred_proba >= threshold).astype(int)
        f1 = f1_score(y_true, y_pred)
        # Calculer les coûts
        fn = np.sum((y_true == 1) & (y_pred == 0))
        fp = np.sum((y_true == 0) & (y_pred == 1))
        cost = fn * 10 + fp
        
        # Sélectionner le seuil basé sur le coût le plus bas
        if cost < best_cost:
            best_f1 = f1
            best_threshold = threshold
            best_cost = cost
    
    return best_f1

In [ ]:


def custom_recall(y_true, y_pred_proba):
    thresholds = np.linspace(0, 1, 100)
    best_threshold = 0.5
    best_recall = 0
    best_cost = float('inf')
    
    for threshold in thresholds:
        y_pred = (y_pred_proba >= threshold).astype(int)
        recall = recall_score(y_true, y_pred)
        # Calculer les coûts
        fn = np.sum((y_true == 1) & (y_pred == 0))
        fp = np.sum((y_true == 0) & (y_pred == 1))
        cost = fn * 10 + fp
        
        # Sélectionner le seuil basé sur le coût le plus bas
        if cost < best_cost:
            best_recall = recall
            best_threshold = threshold
            best_cost = cost
    
    return best_recall

### 2.2.2 - Entrainement

In [ ]:
def run_grid_search(X_train, y_train, model, param_grid):
    # Définir les scorers personnalisés
    scorers = {
        'AUC': 'roc_auc',
        'F1_opt': make_scorer(custom_f1, needs_proba=True),
        'recall_opt': make_scorer(custom_recall, needs_proba=True)
    }

    # Créer un pipeline avec SMOTE et le modèle
    pipeline = Pipeline([
        ('sampling', SMOTE(random_state=42)),
        ('model', model)
    ])

    # Mettre à jour le param_grid pour correspondre au pipeline
    # Les paramètres du modèle doivent être préfixés par 'model__'
    param_grid = {f'model__{key}': value for key, value in param_grid.items()}

    grid_search = GridSearchCV(
        pipeline,
        param_grid=param_grid,
        scoring=scorers,
        refit='AUC',
        cv=2,
        verbose=3,
        return_train_score=True
    )

    # # Fractionnement des données pour alléger le temps de traitement
    # X_train_sample, _, y_train_sample, _ = train_test_split(X_train, y_train, train_size=0.1, random_state=42)

    # Entraînement
    start_time = time.time()
    grid_search.fit(X_train, y_train)
    end_time = time.time()

    execution_time = round(end_time - start_time, 2)
    print(f"Le temps d'execution est de {execution_time} secondes.")

    return {
        'best_model' : grid_search.best_estimator_,
        'best_params_': grid_search.best_params_,
        'best_score_': grid_search.best_score_,
        'cv_results_': grid_search.cv_results_
    }

In [ ]:
# # Configuration du modèle
# model_logit = LogisticRegression(
#     solver='saga', 
#     random_state=42
# )
# model_logit

In [ ]:
# # Paramètres à passer au Grid Search
# param_grid_logit = {
#     'C': np.logspace(-4, 4, 2),
#     'penalty': ['l1', 'l2'],
#     'class_weight': ['balanced', None]
# }
# param_grid_logit

Notes :  

1. **solver='saga'**
Le paramètre solver spécifie l'algorithme à utiliser pour l'optimisation des poids du modèle dans le processus d'apprentissage. saga est une variante de l'algorithme de descente de gradient stochastique qui est bien adaptée pour les grands jeux de données, car elle est efficace en termes de mémoire et supporte les deux types de régularisation L1 ((pour la sélection de caractéristiques) et L2 (pour éviter le surajustement)

2. **max_iter=10000**
Le paramètre max_iter définit le nombre maximal d'itérations que l'algorithme de résolution peut exécuter pour atteindre la convergence. Une valeur élevée comme 10000 assure que l'algorithme a suffisamment de temps pour converger, particulièrement utile dans des scénarios où le modèle a du mal à converger ou lorsque le jeu de données est complexe ou très grand.
Augmenter max_iter permet à l'algorithme plus d'itérations pour trouver les poids optimaux, réduisant ainsi le risque d'arrêter l'apprentissage trop tôt avant que le modèle n'atteigne la stabilité. Bien que permettre plus d'itérations puisse améliorer la précision du modèle, cela peut également augmenter le temps de calcul. Il est donc important de trouver un équilibre entre le nombre d'itérations et les ressources de calcul disponibles.

3. **'F2': make_scorer(fbeta_score, beta=2)**.fbeta_score : Utilise la fonction fbeta_score, qui est une généralisation du F1-score. Le F1-score est en fait un fbeta_score avec beta=1.
beta=2 : Le paramètre beta ajuste l'importance relative du rappel par rapport à la précision. Un beta de 2 donne deux fois plus d'importance au rappel qu'à la précision. Cela peut être utile dans des situations où il est plus critique de minimiser les faux négatifs que les faux positifs.5.


In [ ]:
# # Execution de la fonction
# start_training_time = time.time()
# results_logit = run_grid_search(X_train, y_train, model_logit, param_grid_logit)
# end_training_time = time.time()


In [ ]:
# training_execution_time = round(end_training_time - start_training_time,2)  
# print(f"Le temps d'exécution est de {training_execution_time} secondes.")

In [ ]:
# best_model_logit = results_logit['best_model']
# best_params_logit = results_logit['best_params_']
# best_score_logit = results_logit['best_score_']
# cv_results_logit = results_logit['cv_results_']


# print("Best Params:", best_params_logit)
# print("Best AUC:", best_score_logit)


### 2.2.3 - Analyse de la cross_validation

In [ ]:
def extract_best_auc_result(results):
    """
    Extracts the best model configuration based on AUC score from the cv_results of a GridSearchCV.
    
    Parameters:
    - results (dict): A dictionary output from the run_grid_search function containing 'cv_results_'.
    
    Returns:
    - DataFrame: A DataFrame containing the best scoring row based on the AUC rank.
    """
    # Conversion of cv_results_ into a DataFrame
    cv_results = results['cv_results_']
    df_cv_results = pd.DataFrame(cv_results)
    
    # Sorting the DataFrame by the rank of the AUC test scores
    df_cv_results = df_cv_results.sort_values(by="rank_test_AUC", ascending=True)
    
    # Returning the top entry
    return df_cv_results.head(1)

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# best_auc_result = extract_best_auc_result(results_logit)
# best_auc_result.head(1)


La meilleure combinaison d'hyperparamètres est déterminée en maximisant le F1 score, grâce au paramètre refit='F1_opt' de GridSearchCV. Les autres métriques sont calculées pour fournir une évaluation complète des performances du modèle, mais elles ne sont pas utilisées pour le choix final des hyperparamètres.

In [ ]:
# # Les résultats de la combinaison la mieux classée
# best_auc_result.iloc[0]

On obtient une ligne par combinaison de variables :
- **'mean_fit_time'** et **'std_fit_time'** : Temps moyen et écart-type du temps d'ajustement, respectivement, à travers toutes les folds et les combinaisons de paramètres.  
- **'mean_score_time'** et **'std_score_time'** : Temps moyen et écart-type du temps d'évaluation, respectivement, à travers toutes les folds et les combinaisons de paramètres.
  
- **'param_C'** et **'param_penalty'** etc.. : Les valeurs des hyperparamètres pour chaque combinaison.  
- **'params'** : Liste de dictionnaires représentant les combinaisons de paramètres.
  
- **'split0_test_AUC'**, **'split1_test_AUC'**, ...,  : AUC du test pour chaque fold.  
- **'mean_test_AUC'** et **'std_test_AUC'** : Moyenne et écart-type des AUC à travers toutes les folds pour chaque combinaison de paramètres.  
- **'rank_test_AUC'** : Le classement de chaque combinaison de paramètres basé sur la moyenne des AUC.  


Etc...


=> Dans notre exemple, c'est la combinaison :
{'C': 0.3593813663804626, 'penalty': 'l2'}	
 qui a obtenu la meilleure AUC, comme indiqué par le rang 1 dans les deux cas.

In [ ]:
def plot_auc_scores(best_auc_result):
    """
    Plots the AUC scores from cross-validation folds for the best model configuration based on rank.
    
    Parameters:
    - best_auc_result (DataFrame): A DataFrame with the top row from cv_results_ sorted by AUC.
    - full_cv_results (DataFrame): The full cv_results DataFrame to extract mean scores for rank 1.
    """
    # Extract the AUC scores for each fold for the best AUC configuration
    df_train_results_auc = best_auc_result[[
        'split0_train_AUC', 
        'split1_train_AUC', 
        # 'split2_test_AUC', 
        # 'split3_test_AUC', 
        # 'split4_test_AUC'
    ]][best_auc_result['rank_test_AUC'] == 1].values
    
    df_test_results_auc = best_auc_result[[
        'split0_test_AUC', 
        'split1_test_AUC', 
        # 'split2_test_AUC', 
        # 'split3_test_AUC', 
        # 'split4_test_AUC'
    ]][best_auc_result['rank_test_AUC'] == 1].values
    
    fig, ax = plt.subplots(figsize=(8, 4))
    
    # Plot the train scores
    ax.plot(range(0, 2), df_test_results_auc.reshape(-1), label='Scores de validation')
    
    # Plot the test scores
    ax.plot(range(0, 2), df_train_results_auc.reshape(-1), label='Scores de train')

    # # Plot the average for the combinations with rank_test_AUC == 1
    # mean_scores_rank_1 = best_auc_result.loc[best_auc_result['rank_test_AUC'] == 1, 'mean_test_AUC']
    # ax.axhline(y=mean_scores_rank_1.values[0], color='g', linestyle='--', label='Moyenne (rank 1)')
    
    # Set the properties of the axis
    ax.set_xticks(range(0, 2))
    ax.set_xlabel("Folds de cross-validation")
    ax.set_ylabel('AUC')
    ax.set_title("AUC de chaque fold \n pour la combinaison d'hyperparamètres \n qui arrive au rang 1 en terme d'AUC", fontsize=15, weight="bold", fontname="Impact", color="#0e2452")
    
    # Add a legend
    ax.legend()

    fig.tight_layout()
    
    plt.show()


In [ ]:
# plot_auc_scores(best_auc_result)

### 2.2.4 - Prédictions et métriques d'évaluation

In [ ]:
def evaluate_model (X_train, y_train, X_test, y_test, best_model):

    start_predict_time = time.time()
    # Predict on the training data
    y_train_pred_proba = best_model.predict_proba(X_train)[:, 1]
    
    # Predict on the test data
    y_test_pred_proba = best_model.predict_proba(X_test)[:, 1]

    end_predict_time = time.time()

    # Calculate AUC for training and test data
    auc_train = round(roc_auc_score(y_train, y_train_pred_proba),2)
    auc_test = round(roc_auc_score(y_test, y_test_pred_proba),2)

    # Find the optimal threshold for cost function on the training data
    thresholds = np.linspace(0, 1, 100)
    costs = [calculate_cost_threshold(y_test, y_test_pred_proba, thr, cost_fn=10, cost_fp=1) for thr in thresholds]
    optimal_threshold = round(thresholds[np.argmin(costs)],2) 

    # Metrics at the optimal threshold
    y_test_pred_opt = (y_test_pred_proba >= optimal_threshold).astype(int)
    
    f1_score_1_test = f1_score(y_test, y_test_pred_opt)
    recall_1_test = recall_score(y_test, y_test_pred_opt)
    accuracy_test = accuracy_score(y_test, y_test_pred_opt)
    
    return {
        'predicting_execution_time': round(end_predict_time - start_predict_time, 2),
        'auc_train': auc_train,
        'auc_test': auc_test,
        'optimal_threshold': optimal_threshold,
        'f1_score_1_test': round(f1_score_1_test,2),
        'recall_1_test': round(recall_1_test,2),
        'accuracy_test': round(accuracy_test,2)
    }

In [ ]:
# result_logit=evaluate_model (X_train, y_train, X_test, y_test, best_model_logit)
# result_logit

### 2.2.5 - Enregistrement du modèle dans ML Flow

In [ ]:
# # Création d'une nouvelle expérience MLflow
# mlflow.set_experiment("MLflow Credit_Scoring - Projet_7")

# # Commencer une session MLflow
# with mlflow.start_run():
#     # Log les hyperparameters que j'ai déclarés au dessus
#     mlflow.log_params(best_params_logit)

#     # Log les métriques
    
#     mlflow.log_metric("optimal_threshold", result_logit['optimal_threshold'])

#     mlflow.log_metric("recall_1_test", result_logit['recall_1_test'])
#     mlflow.log_metric("f1_score_1_test", result_logit['f1_score_1_test'])
#     mlflow.log_metric("accuracy du test", result_logit['accuracy_test'])
    
#     mlflow.log_metric("auc_train", result_logit['auc_train'])
#     mlflow.log_metric("auc_test", result_logit['auc_test'])
    
#     mlflow.log_metric("temps_fit", training_execution_time)
#     mlflow.log_metric("temps_predict", result_logit['predicting_execution_time'])

    
 
    

#     # Définir un tag pour se rappeler l'objet de cette session
#     mlflow.set_tag("Training Info", "Logistic Regression with GridSearchCV ")

#     # Infer the model signature
#     signature = infer_signature(X_train, best_model_logit.predict(X_train)) # infer_signature génère automatiquement une "signature" 
#                                                               # qui décrit les entrées et les sorties du modèle. 
#                                                               # Cela inclut les types de données et les formats attendus par le modèle, 
#                                                               # facilitant ainsi l'intégration et la réutilisation du modèle 
#                                                               # dans différents environnements.

    
#     # Log le model
#     model_info = mlflow.sklearn.log_model(
#         sk_model=best_model_logit, # le modèle
#         artifact_path="logistic_regression_model", # le chemin où le modèle sera enregistré
#         signature=signature, # la signature du modèle
#         input_example=X_train, # un exemple d'input pour montrer comment invoquer le modèle
#         registered_model_name="scoring-credit-logistic-regression", # nom sous lequel le modèle est enregistré dans le registre de modèles MLflow
#     )


### 2.4.6 - Feature Importance

In [ ]:
# best_model_logit

In [ ]:
# # extrcation des coefficients
# coefficients_logit=best_model_logit.named_steps['model'].coef_


Pour mémoire, j'avais extrait le nom des features avant imputation des MV et StandardScaling

In [ ]:
# coefficients_logit_dict = dict(zip(features, coefficients_logit.flatten()))
# coefficients_logit=pd.DataFrame([coefficients_logit_dict]).T
# coefficients_logit.columns = ['Importance']
# coefficients_logit_sorted=coefficients_logit.sort_values(by='Importance', ascending=True).tail(10)
# coefficients_logit_sorted

In [ ]:
# plt.figure(figsize=(8, 5))  
# coefficients_logit_sorted.plot(kind='barh', legend=False, ax=plt.gca())  
# plt.title('Top 10 des caractéristiques les plus importantes')  
# plt.xlabel('Importance') 
# plt.ylabel('Caractéristiques')  
# plt.show()

Les 5 premières variables :
- 'AMT_INCOME_TOTAL'	= Revenus du client
- 'AMT_CREDIT' =	Montant du prêt
- 'COMMONAREA_MODE' = Informations normalisées sur l'immeuble où vit le client : taille moyenne (suffixe _AVG), modus (suffixe _MODE), médiane (suffixe _MEDI) de l'appartement, surface commune, surface habitable, âge de l'immeuble, nombre d'ascenseurs, nombre d'entrées, état de l'immeuble, nombre d'étages, etc.
- 'DEF_30_CNT_SOCIAL_CIRCLE' = Cette phrase interroge sur le nombre d'occasions où les entourages sociaux du client ont été enregistrés comme ayant un retard de paiement exactement de 30 jours après la date d'échéance.
- 'ANNUITY_INCOME_PERCENT' = le pourcentage de l'annuité du prêt par rapport au revenu du client

## 2.3 - Random Forest

In [ ]:
# X_sm_sample, _, y_sm_sample, _ = train_test_split(X_sm, y_sm, train_size=0.01, random_state=42)

In [ ]:
# from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import cross_val_score
# from sklearn.datasets import make_classification
# from sklearn.metrics import roc_auc_score


# # Définition de l'espace des hyperparamètres
# space = {
#     'n_estimators': hp.choice('n_estimators', range(10, 200)),
#     'max_depth': hp.choice('max_depth', range(1, 20)),
#     'min_samples_split': hp.uniform('min_samples_split', 0.01, 0.1),
#     'min_samples_leaf': hp.uniform('min_samples_leaf', 0.01, 0.1),
#     'max_features': hp.choice('max_features', ['sqrt', 'log2', None])
# }

# # Fonction objectif à minimiser
# def objective(params):
#     clf = RandomForestClassifier(**params)
#     # Nous utilisons la validation croisée pour estimer l'AUC
#     score = cross_val_score(clf, X_sm_sample, y_sm_sample, scoring='roc_auc', cv=5).mean()
#     # Hyperopt minimise donc nous utilisons -AUC
#     return {'loss': -score, 'status': STATUS_OK}

# # Exécution de la recherche
# trials = Trials()
# best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=100, trials=trials)

# print("Meilleurs hyperparamètres:", best)


###  2.3.1 - Entrainement

In [ ]:
# model_RF = RandomForestClassifier(random_state = 50, 
#                                   class_weight='balanced')
# param_grid_RF = {
#    'n_estimators': [120,150],  
#    'max_depth': [7,8],
#    # 'max_features': [0.0],
#    # 'min_samples_leaf': [0.01,0.05],
#    # 'min_samples_split': [0.03,0.05]
    
# }

In [ ]:
# # Execution de la fonction
# start_training_time = time.time()

# results_RF = run_grid_search(X_train, y_train, model_RF, param_grid_RF)

# end_training_time = time.time()

In [ ]:
# training_execution_time = round(end_training_time - start_training_time,2)  
# print(f"Le temps d'exécution est de {training_execution_time} secondes.")

In [ ]:
# best_model_RF = results_RF['best_model']
# best_params_RF = results_RF['best_params_']
# best_score_RF = results_RF['best_score_']
# cv_results_RF = results_RF['cv_results_']

# print("Best Params:", best_params_RF)
# print("Best AUC:", best_score_RF)

###  2.3.2 - Analyse de la cross_validation

In [ ]:
# best_auc_result_RF = extract_best_auc_result(results_RF)
# best_auc_result_RF.head(1)

In [ ]:
# # Les résultats de la combinaison la mieux classée
# best_auc_result_RF.iloc[0]

In [ ]:
# plot_auc_scores(best_auc_result_RF)

###  2.3.3 - Prédictions et métriques d'évaluation

In [ ]:
# result_RF=evaluate_model(X_train, y_train, X_test, y_test, best_model_RF)
# result_RF

### 2.3.4 - Enregistrement dans ML Flow

In [ ]:
# # Création d'une nouvelle expérience MLflow
# mlflow.set_experiment("MLflow Credit_Scoring - Projet_7")

# # Commencer une session MLflow
# with mlflow.start_run():
#     # Log les hyperparameters que j'ai déclarés au dessus
#     mlflow.log_params(best_params_RF)

#     # Log les métriques
    
#     mlflow.log_metric("optimal_threshold", result_RF['optimal_threshold'])

#     mlflow.log_metric("recall_1_test", result_RF['recall_1_test'])
#     mlflow.log_metric("f1_score_1_test", result_RF['f1_score_1_test'])
#     mlflow.log_metric("accuracy du test", result_RF['accuracy_test'])
    
#     mlflow.log_metric("auc_train", result_RF['auc_train'])
#     mlflow.log_metric("auc_test", result_RF['auc_test'])
    
#     mlflow.log_metric("temps_fit", training_execution_time)
#     mlflow.log_metric("temps_predict", result_RF['predicting_execution_time'])

    
 
    

#     # Définir un tag pour se rappeler l'objet de cette session
#     mlflow.set_tag("Training Info", "Random forest Classifier with GridSearchCV ")

#     # Infer the model signature
#     signature = infer_signature(X_train, best_model_RF.predict(X_train)) # infer_signature génère automatiquement une "signature" 
#                                                               # qui décrit les entrées et les sorties du modèle. 
#                                                               # Cela inclut les types de données et les formats attendus par le modèle, 
#                                                               # facilitant ainsi l'intégration et la réutilisation du modèle 
#                                                               # dans différents environnements.

    
#     # Log le model
#     model_info = mlflow.sklearn.log_model(
#         sk_model=best_model_RF, # le modèle
#         artifact_path="random_forest_classifier_model", # le chemin où le modèle sera enregistré
#         signature=signature, # la signature du modèle
#         input_example=X_train, # un exemple d'input pour montrer comment invoquer le modèle
#         registered_model_name="scoring-credit-random-forest_classifier", # nom sous lequel le modèle est enregistré dans le registre de modèles MLflow
#     )


### 2.3.5 - Feature Importance

In [ ]:
# best_model_RF

In [ ]:
# # extrcation des coefficients
# coefficients_RF=best_model_RF.named_steps['model'].feature_importances_

In [ ]:
# coefficients_RF_dict = dict(zip(features, coefficients_RF.flatten()))
# coefficients_RF=pd.DataFrame([coefficients_RF_dict]).T
# coefficients_RF.columns = ['Importance']
# coefficients_RF_sorted=coefficients_RF.sort_values(by='Importance', ascending=True).tail(10)
# # coefficients_RF_sorted


In [ ]:
# plt.figure(figsize=(8, 5))  
# coefficients_RF_sorted.plot(kind='barh', legend=False, ax=plt.gca())  
# plt.title('Top 10 des caractéristiques les plus importantes')  
# plt.xlabel('Importance') 
# plt.ylabel('Caractéristiques')  
# plt.show()

Les variables les+ influentes :
- 'EXT_SOURCE_3', 'EXT_SOURCE_2', 'EXT_SOURCE_1'	 : Les variables de score normalisé provenant d'une source de données externe sont présentes dans le top 8 des variables les plus influentes
- 'AMT_REQ_CREDIT_BUREAU_YEAR' : Nombre de demandes de renseignements au bureau de crédit concernant le client xxx temps avant la demande (à l'exclusion de XXXX avant la demande)
- 'OBS_60_CNT_SOCIAL_CIRCLE' et 'OBS_30_CNT_SOCIAL_CIRCLE' : Cette phrase cherche à quantifier le nombre d'observations où l'on peut constater que les entourages sociaux du client ont des défauts de paiement dépassant 60 jours et 30 jours après l'échéance prévue.
- 'CNT_CHILDREN' : Nbre d'enfants

Les variables qui ont le plus de poids dans le Random Forest sont très différentes de la Regression logistique

## 2.4 - Light Gradient Boosting Machine

https://machinelearningmastery.com/gentle-introduction-gradient-boosting-algorithm-machine-learning/ 
The LightGBM library : http://lightgbm.readthedocs.io/en/latest/Quick-Start.html

LightGBM est un framework d'apprentissage automatique développé par Microsoft, qui est largement utilisé pour les tâches de classification, de régression et de classement. Il appartient à la catégorie des algorithmes de boosting basés sur des arbres de décision, similaire à XGBoost et CatBoost. Voici quelques points clés pour comprendre LightGBM :

LightGBM utilise le gradient boosting, une technique qui construit de manière itérative des modèles de prévision en ajoutant des arbres de décision faibles, chaque nouvel arbre étant construit pour corriger les erreurs des arbres précédents. Ces arbres sont ajoutés séquentiellement jusqu'à ce qu'un nombre spécifié d'arbres soit atteint ou que les améliorations deviennent négligeables.

**Son fonctionnement** :
1. Initialisation 
- LightGBM commence par créer un modèle initial très simple, souvent appelé "base learner". Ce modèle peut être quelque chose d’aussi simple qu’une régression ou un arbre de décision très basique. L'idée est de créer un point de départ pour l'algorithme.
- Ce premier modèle fait une prédiction basique qui est généralement loin d'être parfaite mais fournit une base sur laquelle construire.
2. Apprendre des Erreurs
- Après avoir fait ses premières prédictions, LightGBM examine les erreurs — les différences entre les valeurs prédites et les valeurs réelles. Ces erreurs sont aussi appelées résidus.
- Un nouveau modèle est ensuite entraîné, mais au lieu de prédire la valeur cible originale, il essaie de prédire ces résidus (les erreurs du premier modèle). L’idée est de corriger là où le premier modèle s’est trompé.
3. Répéter
- Ce processus de correction des erreurs en entraînant un nouveau modèle sur les résidus est répété plusieurs fois. Chaque nouveau modèle est conçu pour améliorer les prédictions là où les modèles précédents ont échoué.
- À chaque étape, l’algorithme se concentre de plus en plus sur les cas les plus difficiles à prédire, raffinant progressivement sa capacité à modéliser les données de manière précise.
4. Combiner
- Tous les modèles simples créés lors des différentes itérations sont combinés pour former le modèle final. Chaque modèle individuel apporte sa contribution spécifique, aidant à prédire une partie différente du problème.
- Dans LightGBM, comme dans d'autres algorithmes de boosting, les contributions de chaque modèle peuvent être pondérées pour optimiser les performances globales. Certains modèles peuvent se voir attribuer plus d'importance en fonction de leur précision.

**Particularités du LightGBM**
- Contrairement à d'autres algorithmes qui construisent des arbres en se développant niveau par niveau, LightGBM utilise une stratégie de croissance par feuilles. Dans cette approche, plutôt que d'ajouter une rangée complète de branches, vous ajoutez la branche la plus nécessaire ou la plus "utile" à chaque étape, indépendamment du niveau. Cela permet de se concentrer plus intensément sur les parties de l'arbre qui réduiront le plus rapidement l'erreur globale. Cela permet souvent de réduire plus rapidement l'erreur globale, bien que le risque de surajustement puisse être légèrement plus élevé, surtout avec des ensembles de données plus petits.
- LightGBM utilise des techniques telles que le partage basé sur des histogrammes pour accélérer l'entraînement. Au lieu de regarder chaque point de donnée unique, il regroupe les valeurs en "bacs" (comme les barres d'un histogramme), et ces bacs sont utilisés pour trouver le meilleur endroit pour diviser les données. Cette méthode réduit considérablement le nombre de calculs nécessaires, car il y a moins de points de division à considérer. Cela rend l'algorithme beaucoup plus rapide et moins gourmand en mémoire, particulièrement avantageux quand on travaille avec de grands ensembles de données.

En résumé, LightGBB utilise une série de modèles qui apprennent progressivement des erreurs des modèles précédents, chacun ajoutant sa propre correction jusqu'à ce qu'un modèle final complet et précis soit formé. C'est une méthode puissante car elle transforme de nombreux apprenants faibles en un prédicteur fort et fiable.

**Avantages** :
- Vitesse et efficacité : L'une des principales forces de LightGBM est sa vitesse d'entraînement rapide et son utilisation efficace de la mémoire.
- Prise en charge des données catégorielles : LightGBM peut gérer directement les caractéristiques catégorielles sans nécessiter de codage préalable, ce qui est une différence significative par rapport à d'autres algorithmes de boosting.
- Facilité d'utilisation et flexibilité : Il est facile à configurer avec une large gamme de paramètres ajustables pour optimiser les performances du modèle.
  
**Applications** :
- LightGBM est utilisé dans une variété de domaines allant de la prédiction de risques financiers à la détection de fraudes, en passant par les recommandations de produits et la prédiction de demandes. Sa capacité à fournir des résultats rapides et précis avec des ensembles de données volumineux le rend particulièrement précieux dans les environnements où les performances et la vitesse sont critiques.



**Quelle différence avec XGboost ?**
LightGBM et XGBoost sont deux des frameworks les plus populaires pour le boosting d'arbres de décision, mais ils présentent des différences clés en termes de méthodologie, d'optimisation, et d'applications typiques. Voici une comparaison des deux, ainsi que les détails sur leur apparition :

XGBoost (eXtreme Gradient Boosting)
Apparition: XGBoost a été développé par Tianqi Chen dans le cadre d'un projet de recherche à l'Université de Washington et a été rendu public en 2014. Il a rapidement gagné en popularité en raison de sa performance dans de nombreuses compétitions de machine learning.

Caractéristiques:
- Utilise un modèle de boosting de gradient, construisant des arbres de manière séquentielle.
- Employe un algorithme basé sur le pré-partitionnement des données en structure de bloc pour gérer les grands ensembles de données de manière plus efficace.
- Propose des régularisations (L1 et L2) pour prévenir le surajustement.
- Utilise la croissance par niveaux des arbres (level-wise), qui équilibre mieux la croissance des arbres mais peut être moins efficace en termes de réduction de l'erreur par rapport à la croissance par feuilles.

LightGBM
Apparition: LightGBM a été développé par Microsoft et introduit en janvier 2017. Il a été conçu pour surmonter certaines des limitations de vitesse et d'échelle de XGBoost et d'autres algorithmes de boosting.

Caractéristiques:
- Utilise également le boosting de gradient mais incorpore plusieurs optimisations pour la vitesse et l'efficacité, comme le partage basé sur des histogrammes.
- Utilise la croissance par feuilles (leaf-wise) qui peut réduire l'erreur plus rapidement mais risque le surajustement sur des petits ensembles de données.
- Gère efficacement les données catégorielles sans prétraitement.
- Optimisé pour la performance en vitesse et en utilisation de mémoire, particulièrement sur des ensembles de données de grande taille.

Différences Clés
- Méthode de construction des arbres : XGBoost construit les arbres niveau par niveau tandis que LightGBM le fait feuille par feuille.
- Gestion des caractéristiques catégorielles : LightGBM peut gérer directement les caractéristiques catégorielles, alors que XGBoost nécessite généralement un prétraitement comme le one-hot encoding.
- Efficacité et vitesse : LightGBM est généralement plus rapide qu'XGBoost, en particulier sur des ensembles de données très grands ou avec de nombreuses caractéristiques, en raison de ses optimisations spécifiques.

Choix entre les deux
Le choix entre XGBoost et LightGBM dépend souvent de la taille de l'ensemble de données et des ressources disponibles :
- XGBoost peut être préférable pour des ensembles de données de taille modérée ou lorsque la prévention du surajustement est critique.
- LightGBM est souvent le choix pour des ensembles de données très volumineux ou pour des applications nécessitant une grande efficacité de calcul.

Dans l'ensemble, les deux frameworks sont hautement performants et le choix entre eux peut aussi dépendre de la familiarité et des préférences personnelles des utilisateurs en matière d'outils d'apprentissage automatique.

### 2.4.1 - Entrainement

In [ ]:
model_LGBM = lgb.LGBMClassifier(random_state = 50)
        
param_grid_LGBM = {
    'num_leaves': [20,25],
    #'learning_rate' : [0.09,0.11], 
     'n_estimators': [100,110]
}

- num_leaves : définit le nombre maximal de feuilles (nœuds terminaux) qu'un arbre peut avoir. Cela détermine la complexité et la capacité d'apprentissage de chaque arbre individuel. Par défaut 31
- learning_rate : contrôle la vitesse à laquelle le modèle apprend. Un taux d'apprentissage, ou learning rate, plus bas signifie que le modèle mettra plus de temps à converger vers une solution optimale, mais cela peut aussi aider à éviter le surapprentissage en faisant des pas plus prudents pendant l'entraînement. Par défaut =0.1
- n_estimators = le nombre d'arbres à construire dans le modèle. par défaut =100

In [ ]:
# Execution de la fonction
start_training_time = time.time()

results_LGBM = run_grid_search(X_train, y_train, model_LGBM, param_grid_LGBM)

end_training_time = time.time()

In [ ]:
training_execution_time = round(end_training_time - start_training_time,2)  
print(f"Le temps d'exécution est de {training_execution_time} secondes.")

In [ ]:
best_model_lgbm = results_LGBM['best_model']
best_params_lgbm = results_LGBM['best_params_']
best_score_lgbm = results_LGBM['best_score_']
cv_results_lgbm = results_LGBM['cv_results_']

print("Best Params:", best_params_lgbm)
print("Best AUC:", best_score_lgbm)

### 2.4.2 - Analyse de la cross_validation

In [ ]:
best_auc_result_LGBM = extract_best_auc_result(results_LGBM)
best_auc_result_LGBM.head(1)

In [ ]:
# Les résultats de la combinaison la mieux classée
best_auc_result_LGBM.iloc[0]

In [ ]:
plot_auc_scores(best_auc_result_LGBM)

### 2.4.3 - Prédictions et métriques d'évaluation

In [ ]:
result_lgbm=evaluate_model (X_train, y_train, X_test, y_test, best_model_lgbm)
result_lgbm

### 2.4.4 - Enregistrement dans ML Flow

In [ ]:
# # Création d'une nouvelle expérience MLflow
# mlflow.set_experiment("MLflow Credit_Scoring - Projet_7")

# # Commencer une session MLflow
# with mlflow.start_run():
#     # Log les hyperparameters que j'ai déclarés au dessus
#     mlflow.log_params(best_params_lgbm)

#     # Log les métriques
    
#     mlflow.log_metric("optimal_threshold", result_lgbm['optimal_threshold'])

#     mlflow.log_metric("recall_1_test", result_lgbm['recall_1_test'])
#     mlflow.log_metric("f1_score_1_test", result_lgbm['f1_score_1_test'])
#     mlflow.log_metric("accuracy du test", result_lgbm['accuracy_test'])
    
#     mlflow.log_metric("auc_train", result_lgbm['auc_train'])
#     mlflow.log_metric("auc_test", result_lgbm['auc_test'])
    
#     mlflow.log_metric("temps_fit", training_execution_time)
#     mlflow.log_metric("temps_predict", result_lgbm['predicting_execution_time'])

    
 
    

#     # Définir un tag pour se rappeler l'objet de cette session
#     mlflow.set_tag("Training Info", "LGBM with GridSearchCV ")

#     # Infer the model signature
#     signature = infer_signature(X_train, best_model_lgbm.predict(X_train)) # infer_signature génère automatiquement une "signature" 
#                                                               # qui décrit les entrées et les sorties du modèle. 
#                                                               # Cela inclut les types de données et les formats attendus par le modèle, 
#                                                               # facilitant ainsi l'intégration et la réutilisation du modèle 
#                                                               # dans différents environnements.

    
#     # Log le model
#     model_info = mlflow.sklearn.log_model(
#         sk_model=best_model_lgbm, # le modèle
#         artifact_path="lgbm_classifier_model", # le chemin où le modèle sera enregistré
#         signature=signature, # la signature du modèle
#         input_example=X_train, # un exemple d'input pour montrer comment invoquer le modèle
#         registered_model_name="scoring-credit-lgbm_classifier", # nom sous lequel le modèle est enregistré dans le registre de modèles MLflow
#     )


### 2.4.5 - Feature importance

In [ ]:
best_model_lgbm

In [ ]:
# extrcation des coefficients
coefficients_lgbm=best_model_lgbm.named_steps['model'].feature_importances_

In [ ]:
coefficients_lgbm_dict = dict(zip(features, coefficients_lgbm.flatten()))
coefficients_lgbm=pd.DataFrame([coefficients_lgbm_dict]).T
coefficients_lgbm.columns = ['Importance']
coefficients_lgbm_sorted=coefficients_lgbm.sort_values(by='Importance', ascending=True).tail(10)
# coefficients_lgbm_sorted


In [ ]:
plt.figure(figsize=(8, 5))  
coefficients_lgbm_sorted.plot(kind='barh', legend=False, ax=plt.gca())  
plt.title('Top 10 des caractéristiques les plus importantes')  
plt.xlabel('Importance') 
plt.ylabel('Caractéristiques')  
plt.show()

- 'HOUR_APPR_PROCESS_START' = À quelle heure environ le client a-t-il demandé le prêt ?
- 'AMT_REQ_CREDIT_BUREAU_YEAR' = Nombre de demandes de renseignements au bureau de crédit concernant le client xxx temps avant la demande (à l'exclusion de XXXX avant la demande)
- 'OBS_30_CNT_SOCIAL_CIRCLE' : Cette phrase cherche à quantifier le nombre d'observations où l'on peut constater que les entourages sociaux du client ont des défauts de paiement dépassant 30 jours après l'échéance prévue.
- 'CREDIT_TERM' : la durée du paiement en mois (puisque l'annuité est le montant mensuel dû).

### 2.4.6 - Interpétation globale

In [ ]:
best_model_lgbm.named_steps['model']

In [ ]:
# Création de l'explainer SHAP pour le modèle LightGBM
best_model_explainer = shap.TreeExplainer(best_model_lgbm.named_steps['model'])

In [ ]:
shap.summary_plot(best_model_explainer.shap_values(X_test),
                  features = X_test,
                  plot_type="bar",
                  feature_names=features,
                   max_display=10,
                   plot_size=(8, 5))



Les 2 classes utilisent les mêmes features de manière à peu près égale. Si on regarde ce qui se passe pour la classe 1 :

In [ ]:
shap.summary_plot(best_model_explainer.shap_values(X_test)[1],
                  features = X_test,
                  feature_names=features,
                   max_display=10,
                   plot_size=(8, 5))

- plus l'entourage de l'emprunteur a des défauts de paeiement de plus de 30 jours, plus le client est risqué
- moins il a d'enfants plus il est risqué
- plus les valeurs d'ext_source 1_2_3 sont faibles, plus le client est risqué

### 2.4.7 - Interpétation locale

In [ ]:
# explainer = shap.TreeExplainer(best_model_lgbm.named_steps['model'])

In [ ]:
# shap_values = explainer.shap_values(X_test)

In [ ]:
# X_test_df = pd.DataFrame(X_test, columns=features)

#### 2.4.7.1 - L'individu le mieux scoré

##### 2.4.7.1.1 - Consolidation du X_test, y_test et y_pred_test

In [ ]:
# X_test_df  = pd.DataFrame(X_test_save, columns=features)
# X_test_df.shape

In [ ]:
# X_test_df.head(1)

In [ ]:
# y_test.shape

In [ ]:
# y_test.head(1)

In [ ]:
# best_model_lgbm.named_steps['model']

In [ ]:
# y_test_pred_proba=best_model_lgbm.named_steps['model'].predict_proba(X_test)[:, 1]

In [ ]:
# len(y_test_pred_proba)

In [ ]:
# # Ajouter y_train et les prédictions au DataFrame
# X_test_df['TARGET'] = y_test
# X_test_df['Predicted_Probability'] = y_test_pred_proba

In [ ]:
# X_test_df.shape

In [ ]:
# X_test_df.head(1)

In [ ]:
# X_test_df[X_test_df['TARGET']==1].head(1)

In [ ]:
# X_test_df_sorted=X_test_df.sort_values(by='Predicted_Probability', ascending=False)


In [ ]:
# X_test_df_sorted.head(2)

# 3 - More Feature Engineering
--------------

Nous allons d'abord nous intéresser au nombre de prêts précédents des clients auprès d'autres institutions financières. 

## 3.1 - Depuis le dataframe "bureau"

In [ ]:
bureau = pd.read_csv('bureaux.csv')
bureau.head()

In [ ]:
bureau.shape

In [ ]:
# Groupby par client id (SK_ID_CURR), comptage du nbre de prêts précédents
previous_loan_counts = bureau.groupby('SK_ID_CURR', as_index=False).agg(previous_loan_counts=('SK_ID_BUREAU','count'))
previous_loan_counts.head()

In [ ]:
previous_loan_counts.shape

In [ ]:
# Ajout de cette colonne au dataframe d'origine (=le train auquel on a ajouté les premières variables métier)
train = app_train_domain.merge(previous_loan_counts, on = 'SK_ID_CURR', how = 'left')
train.shape

In [ ]:
# Imputation des MV par 0
train['previous_loan_counts'] = train['previous_loan_counts'].fillna(0)
train.head()

Pour déterminer si la nouvelle variable est utile, nous pouvons calculer le coefficient de corrélation de Pearson (valeur r) entre cette variable et la cible. Ce coefficient mesure la force d'une relation linéaire entre deux variables et va de -1 (parfaitement linéaire négativement) à +1 (parfaitement linéaire positivement). La valeur r n'est pas la meilleure mesure de l'« utilité » d'une nouvelle variable, mais elle peut donner une première approximation de l'utilité d'une variable pour un modèle d'apprentissage automatique. Plus la valeur r d'une variable est élevée par rapport à la cible, plus une modification de cette variable est susceptible d'affecter la valeur de la cible. Par conséquent, nous recherchons les variables dont la valeur absolue de r est la plus élevée par rapport à la cible.

Nous pouvons également inspecter visuellement une relation avec la cible à l'aide du graphique de l'estimation de la densité du noyau (KDE).

Le KDE Plot montre la distribution d'une seule variable (il s'agit d'un histogramme lissé). Pour voir les différentes distributions dépendant de la valeur d'une variable catégorielle, nous pouvons colorer les distributions différemment en fonction de la catégorie. Par exemple, nous pouvons afficher le KDE du nombre de prêts précédents en fonction de la valeur de TARGET = 1 ou 0. Le KDE résultant montrera toute différence significative dans la distribution de la variable entre les personnes qui n'ont pas remboursé leur prêt (TARGET == 1) et les personnes qui l'ont remboursé (TARGET == 0). Cela peut servir d'indicateur pour déterminer si une variable sera « pertinente » pour un modèle d'apprentissage automatique.

Nous placerons cette fonctionnalité de traçage dans une fonction à réutiliser pour n'importe quelle variable.

In [ ]:
# Plots the disribution of a variable colored by value of the target
def kde_target(var_name, df):
    
    # Calculate the correlation coefficient between the new variable and the target
    corr = df['TARGET'].corr(df[var_name])
    
    # Calculate medians for repaid vs not repaid
    avg_repaid = df.loc[df['TARGET'] == 0, var_name].median()
    avg_not_repaid = df.loc[df['TARGET'] == 1, var_name].median()
    
    plt.figure(figsize = (12, 6))
    
    # Plot the distribution for target == 0 and target == 1
    sns.kdeplot(df.loc[df['TARGET'] == 0, var_name], label = 'TARGET == 0')
    sns.kdeplot(df.loc[df['TARGET'] == 1, var_name], label = 'TARGET == 1')
    
    # label the plot
    plt.xlabel(var_name); plt.ylabel('Density'); plt.title('%s Distribution' % var_name)
    plt.legend();
    
    # print out the correlation
    print('The correlation between %s and the TARGET is %0.4f' % (var_name, corr))
    # Print out average values
    print('Median value for loan that was not repaid = %0.4f' % avg_not_repaid)
    print('Median value for loan that was repaid =     %0.4f' % avg_repaid)

Nous pouvons tester cette fonction en utilisant la variable EXT_SOURCE_3 qui s'est avérée être l'une des variables les plus importantes selon une Random Forest et une Gradient Boosting Machine.

In [ ]:
kde_target('EXT_SOURCE_3', train)

Passons maintenant à la nouvelle variable que nous venons de créer, à savoir le nombre de prêts antérieurs contractés auprès d'autres institutions.

In [ ]:
kde_target('previous_loan_counts', train)

Il est donc difficile de dire si cette variable sera importante. Le coefficient de corrélation est extrêmement faible et il n'y a pratiquement aucune différence notable dans les distributions.

Passons maintenant à la création de quelques autres variables à partir de la base de données du bureau. Nous prendrons la moyenne, la valeur minimale et la valeur maximale de chaque colonne numérique de la base de données Bureau.

### 3.1.1 - Aggregation des variables numériques

Nous allons faire un groupby sur l'ID Client et calculer le 'mean', 'max', 'min', 'sum' des prêts précédents

In [ ]:
numeric_cols = bureau.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols

Mémo : 
SK_ID_CURR	: ID du prêt qui nous intéresse - Un prêt peut avoir plusieurs précédents dans le credit bureau 
SK_BUREAU_ID :	 ID des prêts précédents

In [ ]:
# Construction d'un dictionaire à passer dans agg, à partir des variables numériques, en excluant les ID
agg_dict = {col: ['count', 'mean', 'max', 'min', 'sum'] for col in numeric_cols[2:]}

In [ ]:
agg_dict

In [ ]:
bureau_agg = bureau.drop(columns=['SK_ID_BUREAU']).groupby('SK_ID_CURR', as_index=False).agg(agg_dict)
bureau_agg.head()

Nous devons créer de nouveaux noms pour chacune de ces colonnes. Le code suivant crée de nouveaux noms en ajoutant le statut au nom. Ici, nous devons tenir compte du fait que la base de données a un index à plusieurs niveaux. Je trouve ces index déroutants et difficiles à utiliser, c'est pourquoi j'essaie de les réduire à un index à un seul niveau aussi rapidement que possible.

In [ ]:
# List of column names
columns = ['SK_ID_CURR']

# Iterate through the variables names
for var in bureau_agg.columns.levels[0]:
    # Skip the id name
    if var != 'SK_ID_CURR':
        
        # Iterate through the stat names
        for stat in bureau_agg.columns.levels[1][:-1]:
            # Make a new column name for the variable and stat
            columns.append('bureau_%s_%s' % (var, stat))

In [ ]:
# Assign the list of columns names as the dataframe column names
bureau_agg.columns = columns
bureau_agg.head()

Merge avec le train

In [ ]:
# Merge with the training data
train = train.merge(bureau_agg, on = 'SK_ID_CURR', how = 'left')
train.head()

Nous pouvons calculer la corrélation de toutes les nouvelles valeurs avec la cible. Là encore, nous pouvons utiliser ces valeurs comme une approximation des variables qui peuvent être importantes pour la modélisation.

In [ ]:
# List of new correlations
new_corrs = []

# Iterate through the columns 
for col in columns:
    # Calculate correlation with the target
    corr = train['TARGET'].corr(train[col])
    
    # Append the list as a tuple

    new_corrs.append((col, corr))

In [ ]:
# Tri des corrélations par valeur absolue
# Inversement du tri pour placer les plus grandes valeurs en début de liste
new_corrs = sorted(new_corrs, key = lambda x: abs(x[1]), reverse = True)
new_corrs[:15]

Aucune des nouvelles variables n'a de corrélation significative avec la cible. Nous pouvons examiner le graphique KDE de la variable la plus corrélée, bureau_DAYS_CREDIT_mean, avec la cible en termes de corrélation de magnitude absolue.

In [ ]:
kde_target('bureau_DAYS_CREDIT_mean', train)

DAYS_CREDIT : Combien de jours avant la demande actuelle le client a-t-il demandé un crédit au Credit Bureau ?

Nous constatons une relation positive extrêmement faible entre la moyenne de cette variable et l'objectif, ce qui signifie que les clients qui ont demandé des prêts plus longtemps dans le passé sont potentiellement plus susceptibles de rembourser les prêts auprès de Home Credit. Cependant, avec une corrélation aussi faible, il est tout aussi probable qu'il s'agisse d'un bruit que d'un signal.

Lorsque nous disposons d'un grand nombre de variables, nous nous attendons à ce que certaines d'entre elles soient corrélées par pur hasard, un problème connu sous le nom de comparaisons multiples. Nous pouvons créer des centaines de caractéristiques, et certaines s'avéreront corrélées avec la cible simplement en raison du bruit aléatoire dans les données. Ensuite, lorsque notre modèle s'entraîne, il peut se surajuster à ces variables parce qu'il pense qu'elles ont une relation avec la cible dans l'ensemble d'apprentissage, mais cela ne se généralise pas nécessairement à l'ensemble de test. Il y a de nombreuses considérations à prendre en compte lorsque l'on crée des caractéristiques !

Encapsulons tout le travail précédent dans une fonction. Cela nous permettra de calculer des statistiques agrégées pour des colonnes numériques dans n'importe quel cadre de données. Nous réutiliserons cette fonction lorsque nous voudrons appliquer les mêmes opérations à d'autres cadres de données.

In [ ]:
def agg_numeric(df, parent_var, df_name):
    """
    Groups and aggregates the numeric values in a child dataframe
    by the parent variable.
    
    Parameters
    --------
        df (dataframe): 
            the child dataframe to calculate the statistics on
        parent_var (string): 
            the parent variable used for grouping and aggregating
        df_name (string): 
            the variable used to rename the columns
        
    Return
    --------
        agg (dataframe): 
            a dataframe with the statistics aggregated by the `parent_var` for 
            all numeric columns. Each observation of the parent variable will have 
            one row in the dataframe with the parent variable as the index. 
            The columns are also renamed using the `df_name`. Columns with all duplicate
            values are removed. 
    
    """
    
    # Remove id variables other than grouping variable
    for col in df:
        if col != parent_var and 'SK_ID' in col:
            df = df.drop(columns = col)
            
    # Only want the numeric variables
    parent_ids = df[parent_var].copy()
    numeric_df = df.select_dtypes('number').copy()
    numeric_df[parent_var] = parent_ids

    # Group by the specified variable and calculate the statistics
    agg = numeric_df.groupby(parent_var).agg(['count', 'mean', 'max', 'min', 'sum'])

    # Need to create new column names
    columns = []

    # Iterate through the variables names
    for var in agg.columns.levels[0]:
        if var != parent_var:
            # Iterate through the stat names
            for stat in agg.columns.levels[1]:
                # Make a new column name for the variable and stat
                columns.append('%s_%s_%s' % (df_name, var, stat))
    
    agg.columns = columns
    
    # Remove the columns with all redundant values
    _, idx = np.unique(agg, axis = 1, return_index=True)
    agg = agg.iloc[:, idx]
    
    return agg

Avant de poursuivre, nous pouvons également transformer le code de calcul des corrélations avec la cible en une fonction.

In [ ]:
# Function to calculate correlations with the target for a dataframe
def target_corrs(df):

    # List of correlations
    corrs = []

    # Iterate through the columns 
    for col in df.columns:
        print(col)
        # Skip the target column
        if col != 'TARGET':
            # Calculate correlation with the target
            corr = df['TARGET'].corr(df[col])

            # Append the list as a tuple
            corrs.append((col, corr))
            
    # Sort by absolute magnitude of correlations
    corrs = sorted(corrs, key = lambda x: abs(x[1]), reverse = True)
    
    return corrs

### 3.1.2 - Aggregation des variables catégorielles

Il s'agit de variables discrètes, de sorte que nous ne pouvons pas nous contenter de calculer des statistiques telles que la moyenne et le maximum, qui ne fonctionnent qu'avec des variables numériques. Au lieu de cela, nous nous appuierons sur le calcul du nombre de valeurs de chaque catégorie au sein de chaque variable catégorielle. 

On commence par un One_Hot_Encoding des variables catégorielles

In [ ]:
categorical = pd.get_dummies(bureau.select_dtypes('object'))
categorical['SK_ID_CURR'] = bureau['SK_ID_CURR']
categorical.head()

Pour chacune de ces variables, on va calculer la somme et la moyenne par ID client 

In [ ]:
categorical_grouped = categorical.groupby('SK_ID_CURR').agg(['sum', 'mean'])
categorical_grouped.head()

Les colonnes de somme représentent le nombre de cette catégorie pour le client associé et la moyenne représente le nombre normalisé. 

Nous pouvons utiliser une fonction similaire à la précédente pour renommer les colonnes. Une fois de plus, nous devons gérer l'index à plusieurs niveaux des colonnes. Nous itérons à travers le premier niveau (niveau 0) qui est le nom de la variable catégorielle ajouté à la valeur de la catégorie (à partir de l'encodage à une touche). Ensuite, nous itérons les statistiques que nous avons calculées pour chaque client. Nous renommerons la colonne avec le nom du niveau 0 annexé à la statistique. Par exemple, la colonne avec CREDIT_ACTIF_Actif comme niveau 0 et somme comme niveau 1 deviendra CREDIT_ACTIF_Actif_count.

In [ ]:
categorical_grouped.columns.levels[0][:10]

In [ ]:
categorical_grouped.columns.levels[1]

In [ ]:
group_var = 'SK_ID_CURR'

# Need to create new column names
columns = []

# Iterate through the variables names
for var in categorical_grouped.columns.levels[0]:
    # Skip the grouping variable
    if var != group_var:
        # Iterate through the stat names
        for stat in ['count', 'count_norm']:
            # Make a new column name for the variable and stat
            columns.append('%s_%s' % (var, stat))

#  Rename the columns
categorical_grouped.columns = columns

categorical_grouped.head()

On merge avec le training

In [ ]:
train = train.merge(categorical_grouped, left_on = 'SK_ID_CURR', right_index = True, how = 'left')
train.head()

In [ ]:
train.shape

In [ ]:
train.iloc[:10, 123:]

Pour rendre le code plus efficace, nous pouvons maintenant écrire une fonction pour gérer les variables catégorielles. Cette fonction prendra la même forme que la fonction agg_numeric, c'est-à-dire qu'elle acceptera un cadre de données et une variable de regroupement. Elle calculera ensuite les effectifs et les effectifs normalisés de chaque catégorie pour toutes les variables catégorielles de la base de données.

In [ ]:
def agg_categorical(df, parent_var, df_name):
    """
    Aggregates the categorical features in a child dataframe
    for each observation of the parent variable.
    
    Parameters
    --------
    df : dataframe 
        The dataframe to calculate the value counts for.
        
    parent_var : string
        The variable by which to group and aggregate the dataframe. For each unique
        value of this variable, the final dataframe will have one row
        
    df_name : string
        Variable added to the front of column names to keep track of columns

    
    Return
    --------
    categorical : dataframe
        A dataframe with aggregated statistics for each observation of the parent_var
        The columns are also renamed and columns with duplicate values are removed.
        
    """
    
    # Select the categorical columns
    categorical = pd.get_dummies(df.select_dtypes(['object', 'category']))

    # Make sure to put the identifying id on the column
    categorical[parent_var] = df[parent_var]

    # Groupby the group var and calculate the sum and mean
    categorical = categorical.groupby(parent_var).agg(['sum', 'count', 'mean'])
    
    column_names = []
    
    # Iterate through the columns in level 0
    for var in categorical.columns.levels[0]:
        # Iterate through the stats in level 1
        for stat in ['sum', 'count', 'mean']:
            # Make a new column name
            column_names.append('%s_%s_%s' % (df_name, var, stat))
    
    categorical.columns = column_names
    
    # Remove duplicate columns by values
    _, idx = np.unique(categorical, axis = 1, return_index = True)
    categorical = categorical.iloc[:, idx]
    
    return categorical

## 3.2 - Depuis le dataframe "bureau_balance"

Nous allons maintenant nous pencher sur la base de données 'bureau_balance'.   
Ce dataframe contient des informations mensuelles sur le(s) prêt(s) précédent(s) de chaque client auprès d'autres institutions financières. Au lieu de regrouper ce dataframe par **SK_ID_CURR (identifiant du client)**, nous allons d'abord regrouper le dataframe par **SK_ID_BUREAU (identifiant du prêt précédent)**. 
Nous obtiendrons ainsi une ligne du dataframe pour chaque prêt. 
Ensuite, nous pourrons regrouper par SK_ID_CURR et calculer les agrégations pour les prêts de chaque client. 
Le résultat final sera un dataframe avec une ligne pour chaque client, avec des statistiques calculées pour leurs prêts.

In [ ]:
bureau_balance = pd.read_csv('bureau_balances.csv')
bureau_balance.head()

**MONTHS_BALANCE**=Mois du solde par rapport à la date d'application (-1 signifie la date de solde la plus récente)  

**STATUS**= Statut du prêt du Bureau de Crédit durant le mois (actif, clôturé, DPD0-30, ... [C signifie clôturé, X signifie statut inconnu, 0 signifie pas de DPD, 1 signifie le maximum fait durant le mois entre 1-30, 2 signifie DPD 31-60, ... 5 signifie DPD 120+ ou vendu ou radié])".
Cette phrase décrit les différents statuts possibles d'un prêt rapporté au Bureau de Crédit pendant un mois donné, incluant des informations sur les jours de retard de paiement (DPD - Days Past Due)

Tout d'abord, nous pouvons calculer le nombre de valeurs de chaque statut pour chaque prêt. Pour ça, on va réutiliser notre fonction

In [ ]:
# Counts of each type of status for each previous loan
bureau_balance_counts = agg_categorical(bureau_balance, 'SK_ID_BUREAU', 'bureau_balance')
bureau_balance_counts.head()

La colonne MONTHS_BALANCE contient les « mois de solde par rapport à la date de la demande ». Cette variable numérique n'est pas nécessairement très importante et, dans le cadre de travaux futurs, nous pourrions la considérer comme une variable temporelle. Pour l'instant, nous pouvons simplement calculer les mêmes statistiques d'agrégation que précédemment.

In [ ]:
# Calculate value count statistics for each `SK_ID_CURR` 
bureau_balance_agg = agg_numeric(bureau_balance, 'SK_ID_BUREAU', df_name = 'bureau_balance')
bureau_balance_agg.head()

Les dataframe ci-dessus contiennent les calculs effectués pour chaque prêt. Nous devons maintenant les agréger pour chaque client. Nous pouvons le faire en fusionnant d'abord les dataframes, puis, comme toutes les variables sont numériques, il nous suffit d'agréger à nouveau les statistiques, en les regroupant cette fois par SK_ID_CURR.

In [ ]:
# Dataframe grouped by the loan
bureau_by_loan = bureau_balance_agg.merge(bureau_balance_counts, right_index = True, left_on = 'SK_ID_BUREAU', how = 'outer')

# Merge to include the SK_ID_CURR
bureau_by_loan = bureau_by_loan.merge(bureau[['SK_ID_BUREAU', 'SK_ID_CURR']], on = 'SK_ID_BUREAU', how = 'left')

bureau_by_loan.head()

In [ ]:
bureau_balance_by_client = agg_numeric(bureau_by_loan.drop(columns = ['SK_ID_BUREAU']), 'SK_ID_CURR', df_name = 'client')
bureau_balance_by_client.head()

Pour résumer, pour le dataframe bureau_balance, nous avons:
- calculé les statistiques numériques en les regroupant par prêt
- comptabilisé les valeurs de chaque variable catégorielle en les regroupant par prêt.
- Fusionné les statistiques et les comptages de valeurs sur les prêts
- Calculé les statistiques numériques pour la base de données résultante, en les regroupant par identifiant de client.

Le dataframe final comporte une ligne pour chaque client, avec des statistiques calculées pour tous leurs prêts avec des informations sur le solde mensuel.

Certaines de ces variables sont un peu confuses, alors essayons d'en expliquer quelques-unes :
- client_bureau_balance_MONTHS_BALANCE_mean_mean : Pour chaque prêt, calcul de la valeur moyenne de MONTHS_BALANCE. Ensuite, pour chaque client, calcul de la moyenne de cette valeur pour tous ses prêts.
- client_bureau_balance_STATUS_X_count_norm_sum : Pour chaque prêt, calcul du nombre d'occurrences de STATUS == X divisé par le nombre de valeurs STATUS totales pour le prêt. Ensuite, pour chaque client, addition des valeurs pour chaque prêt.
  
Nous ne calculerons pas les corrélations tant que nous n'aurons pas rassemblé toutes les variables dans un seul dataframe

Nous disposons à présent de tous les éléments nécessaires pour rassembler les informations relatives aux prêts antérieurs contractés auprès d'autres institutions et les informations relatives aux paiements mensuels de ces prêts dans le dataframe principal de training. 


In [ ]:
# Merge with the monthly information grouped by client
train = train.merge(bureau_balance_by_client, on = 'SK_ID_CURR', how = 'left')

In [ ]:
new_features = list(train.columns)
print("Nombre de variables utilisant les prêts précédents passés dans d'autres institutions financières: ", len(new_features))

# 4  - Analyse des nouvelles variables et sélection 

Nous voulons maintenant examiner les variables que nous avons créées. Nous pouvons examiner :
- le pourcentage de valeurs manquantes,
- les corrélations des variables avec la cible,
- ainsi que la corrélation des variables avec les autres variables.

Les corrélations entre les variables peuvent montrer si nous avons des variables colinéaires, c'est-à-dire des variables fortement corrélées entre elles. Souvent, nous voulons supprimer une variable dans une paire de variables colinéaires parce que les deux variables seraient redondantes.

Nous pouvons également utiliser le pourcentage de valeurs manquantes pour supprimer les caractéristiques dont la majorité des valeurs sont absentes. La sélection des caractéristiques sera un point important à l'avenir, car la réduction du nombre de caractéristiques peut aider le modèle à apprendre pendant la formation et à mieux se généraliser aux données de test. La « malédiction de la dimensionnalité » est le nom donné aux problèmes causés par un trop grand nombre de caractéristiques (une dimension trop élevée). Lorsque le nombre de variables augmente, le nombre de points de données nécessaires pour apprendre la relation entre ces variables et la valeur cible augmente de façon exponentielle.

La sélection des caractéristiques est le processus qui consiste à supprimer des variables pour aider notre modèle à apprendre et à mieux se généraliser à l'ensemble de test. L'objectif est de supprimer les variables inutiles/redondantes tout en préservant celles qui sont utiles. Il existe un certain nombre d'outils que nous pouvons utiliser pour ce processus, mais dans ce NB nous nous en tiendrons à la suppression des colonnes ayant un pourcentage élevé de valeurs manquantes et des variables qui ont une forte corrélation entre elles. 

Plus tard, nous pourrons utiliser les features Importance renvoyées par des modèles tels que Gradient Boosting Machine ou Random Forest pour effectuer la sélection des caractéristiques.

## 4.1 - MV

In [ ]:
# Function to calculate missing values by column# Funct 
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [ ]:
missing_train = missing_values_table(train)
missing_train.head(10)

Nous constatons qu'un certain nombre de colonnes présentent un pourcentage élevé de valeurs manquantes. Il n'existe pas de seuil bien établi pour la suppression des valeurs manquantes, et la meilleure ligne de conduite dépend du problème. Ici, pour réduire le nombre de caractéristiques, nous supprimons toutes les colonnes des données d'apprentissage ou de test qui comportent plus de 90 % de valeurs manquantes.

In [ ]:
missing_train_vars = list(missing_train.index[missing_train['% of Total Values'] > 90])
len(missing_train_vars)

In [ ]:
train = train.drop(columns = missing_train_vars)

Nous n'avons finalement supprimé aucune colonne car il n'y a pas de colonnes avec plus de 90 % de valeurs manquantes. Il se peut que nous devions appliquer une autre méthode de sélection des caractéristiques pour réduire la dimensionnalité.

Faisons une fonction pour plus tard

In [ ]:
def remove_missing_columns(train, threshold = 90):
    # Calculate missing stats for train  (remember to calculate a percent!)
    train_miss = pd.DataFrame(train.isnull().sum())
    train_miss['percent'] = 100 * train_miss[0] / len(train)
    
    # list of missing columns for train and test
    missing_columns = list(train_miss.index[train_miss['percent'] > threshold])
    
    # Print information
    print('There are %d columns with greater than %d%% missing values.' % (len(missing_columns), threshold))
    
    # Drop the missing columns and return
    train = train.drop(columns = missing_columns)
    
    return train

In [ ]:
train.to_csv('train_bureau_raw.csv', index = False)

## 4.2 - Corrélation

### 4.2.1 - Corrélation avec la target

Regardons d'abord les corrélations des variables avec la cible. 

In [ ]:
numeric_columns=train.select_dtypes(include=np.number).columns.tolist()

In [ ]:
corrs = train[numeric_columns].corr()

In [ ]:
corrs = corrs.sort_values('TARGET', ascending = False)

# Ten most positive correlations
pd.DataFrame(corrs['TARGET'].head(10))

In [ ]:
# Ten most negative correlations
pd.DataFrame(corrs['TARGET'].dropna().tail(10))

La variable la plus corrélée avec la cible est une variable que nous avons créée. Cependant, ce n'est pas parce que la variable est corrélée qu'elle sera utile, et nous devons nous rappeler que si nous générons des centaines de nouvelles variables, certaines seront corrélées avec la cible simplement à cause du bruit aléatoire.

Si l'on considère les corrélations avec scepticisme, il apparaît que plusieurs des variables nouvellement créées peuvent être utiles. Pour évaluer l'« utilité » des variables, nous examinerons les importances des caractéristiques renvoyées par le modèle. 

### 4.2.2 - Variables colinéaires

Cherchons toutes les variables qui ont une corrélation supérieure à 0,8 avec d'autres variables.

In [ ]:
# Set the threshold
threshold = 0.8

# Empty dictionary to hold correlated variables
above_threshold_vars = {}

# For each column, record the variables that are above the threshold
for col in corrs:
    above_threshold_vars[col] = list(corrs.index[corrs[col] > threshold])

Pour chacune de ces paires de variables fortement corrélées, nous ne voulons supprimer qu'une seule des variables. Le code suivant crée un ensemble de variables à supprimer en n'ajoutant qu'une seule variable par paire.

In [ ]:
# Track columns to remove and columns already examined
cols_to_remove = []
cols_seen = []
cols_to_remove_pair = []

# Iterate through columns and correlated columns
for key, value in above_threshold_vars.items():
    # Keep track of columns already examined
    cols_seen.append(key)
    for x in value:
        if x == key:
            next
        else:
            # Only want to remove one in a pair
            if x not in cols_seen:
                cols_to_remove.append(x)
                cols_to_remove_pair.append(key)
            
cols_to_remove = list(set(cols_to_remove))
print('Number of columns to remove: ', len(cols_to_remove))

Nous pouvons supprimer ces colonnes du train. Nous devrons comparer la performance après avoir supprimé ces variables avec la performance en conservant ces variables (les fichiers csv bruts que nous avons sauvegardés plus tôt).

In [ ]:
train_corrs_removed = train.drop(columns = cols_to_remove)

print('Training Corrs Removed Shape: ', train_corrs_removed.shape)


In [ ]:
train_corrs_removed.to_csv('train_bureau_corrs_removed.csv', index = False)

# 5 - More Feature Engineering (2)

## 5.1 - Depuis le dataframe 'previous_application'

In [ ]:
previous = pd.read_csv('previous_applications.csv')
previous.head()

Avant d'aller plus loin, on va créer une fonction pour convertir les types de variables. Cela permettra de réduire l'utilisation de la mémoire en utilisant des types plus efficaces pour les variables. Par exemple, la catégorie est souvent un meilleur type que l'objet (sauf si le nombre de catégories uniques est proche du nombre de lignes dans le cadre de données).

In [ ]:
import sys

def return_size(df):
    """Return size of dataframe in gigabytes"""
    return round(sys.getsizeof(df) / 1e9, 2)

def convert_types(df, print_info = False):
    
    original_memory = df.memory_usage().sum()
    
    # Iterate through each column
    for c in df:
        
        # Convert ids and booleans to integers
        if ('SK_ID' in c):
            df[c] = df[c].fillna(0).astype(np.int32)
            
        # Convert objects to category
        elif (df[c].dtype == 'object') and (df[c].nunique() < df.shape[0]):
            df[c] = df[c].astype('category')
        
        # Booleans mapped to integers
        elif list(df[c].unique()) == [1, 0]:
            df[c] = df[c].astype(bool)
        
        # Float64 to float32
        elif df[c].dtype == float:
            df[c] = df[c].astype(np.float32)
            
        # Int64 to int32
        elif df[c].dtype == int:
            df[c] = df[c].astype(np.int32)
        
    new_memory = df.memory_usage().sum()
    
    if print_info:
        print(f'Original Memory Usage: {round(original_memory / 1e9, 2)} gb.')
        print(f'New Memory Usage: {round(new_memory / 1e9, 2)} gb.')
        
    return df

In [ ]:
previous = convert_types(previous, print_info=True)
previous.head()

In [ ]:
# Calculate aggregate statistics for each numeric column
previous_agg = agg_numeric(previous, 'SK_ID_CURR', 'previous')
print('Previous aggregation shape: ', previous_agg.shape)
previous_agg.head()

In [ ]:
# Calculate value counts for each categorical column
previous_counts = agg_categorical(previous, 'SK_ID_CURR', 'previous')
print('Previous counts shape: ', previous_counts.shape)
previous_counts.head()

Nous pouvons joindre la base de données calculée à la base de données d'apprentissage principale à l'aide d'une fusion. Nous devons ensuite supprimer les dataframes calculés pour éviter d'utiliser une trop grande partie de la mémoire du noyau.

In [ ]:
# Merge in the previous information
train = train.merge(previous_counts, on ='SK_ID_CURR', how = 'left')
train = train.merge(previous_agg, on = 'SK_ID_CURR', how = 'left')


# Remove variables to free memory
gc.enable()
del previous, previous_agg, previous_counts
gc.collect()

Suppresion des variables dont les MV sont >90%

In [ ]:
train = remove_missing_columns(train)

Fonction d'agrégation des statistiques au niveau du client

In [ ]:
def aggregate_client(df, group_vars, df_names):
    """Aggregate a dataframe with data at the loan level at the client level
    
    Args:
        df (dataframe): data at the loan level
        group_vars (list of two strings): grouping variables for the loan and then the client
                                          (example ['SK_ID_PREV', 'SK_ID_CURR'])
        df_names (list of two strings): names to call the resulting columns
                                        (example ['cash', 'client'])
        
    Returns:
        df_client (dataframe): aggregated numeric stats at the client level. 
                               Each client will have a single row with all the numeric data aggregated
    """
    
    # Assume agg_numeric and agg_categorical are defined elsewhere, adjust as necessary

    # Aggregate the numeric columns
    df_agg = agg_numeric(df, parent_var=group_vars[0], df_name=df_names[0])
    
    # Check if there are categorical or object type variables
    if any(df.dtypes.isin(['category', 'object'])):
    
        # Count the categorical columns
        df_counts = agg_categorical(df, parent_var=group_vars[0], df_name=df_names[0])

        # Merge the numeric and categorical
        df_by_loan = df_counts.merge(df_agg, on=group_vars[0], how='outer')

        gc.enable()
        del df_agg, df_counts
        gc.collect()

        # Merge to get the client id in dataframe
        df_by_loan = df_by_loan.merge(df[[group_vars[0], group_vars[1]]], on=group_vars[0], how='left')

        # Remove the loan id
        df_by_loan = df_by_loan.drop(columns=[group_vars[0]])

        # Aggregate numeric stats by column
        df_by_client = agg_numeric(df_by_loan, parent_var=group_vars[1], df_name=df_names[1])

    # No categorical or object variables
    else:
        # Merge to get the client id in dataframe
        df_by_loan = df_agg.merge(df[[group_vars[0], group_vars[1]]], on=group_vars[0], how='left')
        
        gc.enable()
        del df_agg
        gc.collect()
        
        # Remove the loan id
        df_by_loan = df_by_loan.drop(columns=[group_vars[0]])
        
        # Aggregate numeric stats by column
        df_by_client = agg_numeric(df_by_loan, parent_var=group_vars[1], df_name=df_names[1])
        
    # Memory management
    gc.enable()
    del df, df_by_loan
    gc.collect()

    return df_by_client


## 5.2 - Depuis le dataframe  POS_CASH_balance

In [ ]:
cash = pd.read_csv('POS_CASH_balances.csv')

cash.head()

In [ ]:
cash = convert_types(cash, print_info=True)
cash.head()

In [ ]:
cash_by_client = aggregate_client(cash, ['SK_ID_PREV', 'SK_ID_CURR'], df_names = ['cash', 'client'])
cash_by_client.head()

In [ ]:
print('Cash by Client Shape: ', cash_by_client.shape)
train = train.merge(cash_by_client, on = 'SK_ID_CURR', how = 'left')

gc.enable()
del cash, cash_by_client
gc.collect()

In [ ]:
train = remove_missing_columns(train)

## 5.3 - Depuis le dataframe  credit_card_balance

In [ ]:
credit = pd.read_csv('credit_card_balances.csv')
credit.head()

In [ ]:
credit = convert_types(credit, print_info = True)
credit.head()

In [ ]:
credit_by_client = aggregate_client(credit, group_vars = ['SK_ID_PREV', 'SK_ID_CURR'], df_names = ['credit', 'client'])
credit_by_client.head()

In [ ]:
print('Credit by client shape: ', credit_by_client.shape)

train = train.merge(credit_by_client, on = 'SK_ID_CURR', how = 'left')

gc.enable()
del credit, credit_by_client
gc.collect()

In [ ]:
train = remove_missing_columns(train)

## 5.4 - Depuis le dataframe 'installments_payments'

In [ ]:
installments = pd.read_csv('installments_payment.csv')
installments.head()

In [ ]:
installments = convert_types(installments, print_info = True)
installments.head()

In [ ]:
installments_by_client = aggregate_client(installments, group_vars = ['SK_ID_PREV', 'SK_ID_CURR'], df_names = ['installments', 'client'])
installments_by_client.head()

In [ ]:
print('Installments by client shape: ', installments_by_client.shape)

train = train.merge(installments_by_client, on = 'SK_ID_CURR', how = 'left')
gc.enable()
del installments, installments_by_client
gc.collect()

In [ ]:
train= remove_missing_columns(train)

In [ ]:
print('Final Training Shape: ', train.shape)

In [ ]:
print(f'Final training size: {return_size(train)}')

# 6 - Feature Selection

# 7 - Nouvelle modélisation

Faisons d'abord une fonction pour toute la partie de préparation

In [ ]:
def prepare_modelisation (df) :

    X = df.drop(['SK_ID_CURR','TARGET'], axis=1)  
    y = df['TARGET']  
    
    # split train/test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Create a label encoder object
    le = LabelEncoder()
    le_count = 0
    
    # Iterate through the columns
    for col in df:
        if df[col].dtype == 'object':
            # If 2 or fewer unique categories
            if len(list(df[col].unique())) <= 2:
                # Train on the training data
                le.fit(df[col])
                # Transform  training, val and testing data
                X_train[col] = le.transform(X_train[col])
                X_test[col] = le.transform(X_test[col])
                            
                # Keep track of how many columns were label encoded
                le_count += 1
                
    # one-hot encoding of categorical variables
    X_train = pd.get_dummies(X_train)
    X_test = pd.get_dummies(X_test)
    
    
    # Median imputation of missing values
    imputer = SimpleImputer(strategy = 'median')
    
    # Scale each feature to 0-1
    scaler = MinMaxScaler(feature_range = (0, 1))
    
    # Fit on the training data
    imputer.fit(X_train)
    
    # Transform both training and testing data
    X_train = imputer.transform(X_train)
    X_test = imputer.transform(X_test)
    
    # Repeat with the scaler
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    
    return X_train, X_test, y_train, y_test

## 7.1 - Préparation du dataframe à la  modélisation 

In [ ]:
X_train, X_test, y_train, y_test=prepare_modelisation(train)

In [ ]:
X_train.shape, X_test.shape,y_train.shape, y_test.shape

## 7.2 - Training 

In [ ]:
model_LGBM = lgb.LGBMClassifier(random_state = 50)
        
param_grid_LGBM = {
    'num_leaves': [20,25],
    # 'learning_rate' : [0.09,0.11], 
    # 'n_estimators': [100,110]
}

In [ ]:
# Execution de la fonction
start_training_time = time.time()

results_LGBM = run_grid_search(X_train, y_train, model_LGBM, param_grid_LGBM)

end_training_time = time.time()

In [ ]:
training_execution_time = round(end_training_time - start_training_time,2)  
print(f"Le temps d'exécution est de {training_execution_time} secondes.")

In [ ]:
best_model_lgbm = results_LGBM['best_model']
best_params_lgbm = results_LGBM['best_params_']
best_score_lgbm = results_LGBM['best_score_']
cv_results_lgbm = results_LGBM['cv_results_']

print("Best Params:", best_params_lgbm)
print("Best AUC:", best_score_lgbm)

### 4.1.3 - Prédictions et évaluation

In [ ]:
result_lgbm=evaluate_model (X_train, y_train, X_test, y_test, best_model_lgbm)
result_lgbm

__Follow-up Notebooks__

For those looking to keep working on this problem, I have a series of follow-up notebooks:

* [Introduction to Automated Feature Engineering](https://www.kaggle.com/willkoehrsen/automated-feature-engineering-basics)
* [Advanced Automated Feature Engineering](https://www.kaggle.com/willkoehrsen/tuning-automated-feature-engineering-exploratory)
* [Feature Selection](https://www.kaggle.com/willkoehrsen/introduction-to-feature-selection)
* [Intro to Model Tuning: Grid and Random Search](https://www.kaggle.com/willkoehrsen/intro-to-model-tuning-grid-and-random-search)
* [Automated Model Tuning](https://www.kaggle.com/willkoehrsen/automated-model-tuning)
* [Model Tuning Results](https://www.kaggle.com/willkoehrsen/model-tuning-results-random-vs-bayesian-opt/notebook)




### Follow-up Notebooks

For those looking to keep working on this problem, I have a series of follow-up notebooks:

* [Manual Feature Engineering Part One](https://www.kaggle.com/willkoehrsen/introduction-to-manual-feature-engineering)
* [Manual Feature Engineering Part Two](https://www.kaggle.com/willkoehrsen/introduction-to-manual-feature-engineering-p2)
* [Introduction to Automated Feature Engineering](https://www.kaggle.com/willkoehrsen/automated-feature-engineering-basics)
* [Advanced Automated Feature Engineering](https://www.kaggle.com/willkoehrsen/tuning-automated-feature-engineering-exploratory)
* [Feature Selection](https://www.kaggle.com/willkoehrsen/introduction-to-feature-selection)
* [Intro to Model Tuning: Grid and Random Search](https://www.kaggle.com/willkoehrsen/intro-to-model-tuning-grid-and-random-search)

As always, I welcome feedback and constructive criticism. I write for Towards Data Science at https://medium.com/@williamkoehrsen/ and can be reached on Twitter at https://twitter.com/koehrsen_will

Will



+ faire woe ?
+ shapley
+ poids de chaque variable
+ récupération de la p value de chaque variable 
+ techniques de rééchantillonage
+ Faire courbe aucroc
+ Faire courbe Lift
+ faire taux de cible par décile